In [1]:
import cv2 
import os 
import numpy as np 
from random import shuffle 
from tqdm import tqdm
import tflearn 
from tflearn.layers.conv import conv_2d, max_pool_2d 
from tflearn.layers.core import input_data, dropout, fully_connected 
from tflearn.layers.estimator import regression 
from keras import Sequential
from keras import models
from keras import layers
from keras.layers import *
from sklearn.model_selection import *
from sklearn.metrics import *

os.getcwd()
os.chdir("/home/aniketh/Documents/fog-parkinsons/rps/120s/individual_data")
data_sub_1 = np.load("data_sub_1.npy")
data_sub_2 = np.load("data_sub_2.npy")
data_sub_3 = np.load("data_sub_3.npy")
data_sub_4 = np.load("data_sub_4.npy")
data_sub_5 = np.load("data_sub_5.npy")
data_sub_6 = np.load("data_sub_6.npy")
data_sub_7 = np.load("data_sub_7.npy")
data_sub_8 = np.load("data_sub_8.npy")
data_sub_9 = np.load("data_sub_9.npy")
data_sub_10 = np.load("data_sub_10.npy")

labels_sub_1 = np.load("labels_sub_1.npy")
labels_sub_2 = np.load("labels_sub_2.npy")
labels_sub_3 = np.load("labels_sub_3.npy")
labels_sub_4 = np.load("labels_sub_4.npy")
labels_sub_5 = np.load("labels_sub_5.npy")
labels_sub_6 = np.load("labels_sub_6.npy")
labels_sub_7 = np.load("labels_sub_7.npy")
labels_sub_8 = np.load("labels_sub_8.npy")
labels_sub_9 = np.load("labels_sub_9.npy")
labels_sub_10 = np.load("labels_sub_10.npy")

Using TensorFlow backend.


## Subject 1 as test

In [6]:
data_training = np.concatenate((data_sub_2, data_sub_3, data_sub_4, data_sub_5,
                                data_sub_6, data_sub_7, data_sub_8 , data_sub_9, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_2, labels_sub_3, labels_sub_4, labels_sub_5, 
                                  labels_sub_6, labels_sub_7, labels_sub_8, labels_sub_9, labels_sub_10))
unique_elements, counts_elements = np.unique(labels_training, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))

    
IMG_SIZE = 100
model = Sequential()

model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(100, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(50, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(1, activation = "softmax"))

model.compile(
 optimizer = "adam",
 loss = "binary_crossentropy",
 metrics = ["accuracy"]
)

model.fit(x=data_training, y=labels_training, batch_size=20, epochs=1,
     verbose=1, shuffle=True)
loss, acc = model.evaluate(x=data_sub_1, y=labels_sub_1, batch_size=20, verbose=1)

print(loss, acc)

Frequency of unique values of the said array:
[[   0    1]
 [1976 1462]]
Epoch 1/1
517/517 [==============================] - 2s 5ms/step
8.048283254154896 0.49516441051918486


In [9]:
sens_l = []
spec_l = []
acc_l = []

data_training = np.concatenate((data_sub_2, data_sub_3, data_sub_4, data_sub_5,
                                data_sub_6, data_sub_7, data_sub_8 , data_sub_9, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_2, labels_sub_3, labels_sub_4, labels_sub_5, 
                                  labels_sub_6, labels_sub_7, labels_sub_8, labels_sub_9, labels_sub_10))


def load_model():
    
    IMG_SIZE = 100
    model = Sequential()

    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(100, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation = "sigmoid"))

    model.compile(
     optimizer = "adam",
     loss = "binary_crossentropy",
     metrics = ["accuracy"]
    )
    return model

def train_evaluate(model, data_train, labels_train, data_test, labels_test):
    model.fit(x=data_train, y=labels_train, batch_size=5, epochs=1,
         verbose=1, shuffle=True)
    loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)
    print(loss, acc)
    y_pred = model.predict(data_test, verbose=0)
    y_pred = y_pred.round()
    y_pred = np.array([int(i) for i in y_pred])
    tn, fp, fn, tp = confusion_matrix(labels_test,y_pred).ravel()
    sens = tp/(tp+fn)
    sens_l.append(sens)
    spec = tn/(tn+fp)
    spec_l.append(spec)
    acc_l.append(acc)
    print(confusion_matrix(labels_test,y_pred))
    print("sens and spec is:")
    print(sens, spec)
    
from sklearn.model_selection import StratifiedKFold

if __name__ == "__main__":
    load_model()
    kFold = StratifiedKFold(n_splits=5, shuffle = True)
    for train, test in kFold.split(data_training,labels_training):
        model = None
        model = load_model()
        train_evaluate(model, data_training[train], labels_training[train], data_training[test], labels_training[test])

        
    loss, acc = model.evaluate(x=data_sub_1, y=labels_sub_1, batch_size=5, verbose=1)
    y_pred = model.predict(data_sub_1, verbose=0)
    y_pred = y_pred.round()
    y_pred = np.array([int(i) for i in y_pred])
    tn, fp, fn, tp = confusion_matrix(labels_sub_1,y_pred).ravel()
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)
    print("sensitivity, specificity and accuracy of models:")
    print(np.mean(sens_l))
    print(np.mean(spec_l))
    print(np.mean(acc_l))
    print(confusion_matrix(labels_sub_1,y_pred))
    print("sensitivity, specificity and accuracy of subject:")
    print(sens)
    print(spec)
    print(acc)

Epoch 1/1
689/689 [==============================] - 4s 5ms/step
0.24818043948214702 0.8722786707438648
[[317  79]
 [  9 284]]
sens and spec is:
0.9692832764505119 0.8005050505050505
Epoch 1/1
688/688 [==============================] - 4s 5ms/step
0.22870537904718943 0.9229651194628935
[[380  15]
 [ 38 255]]
sens and spec is:
0.8703071672354948 0.9620253164556962
Epoch 1/1
687/687 [==============================] - 4s 5ms/step
0.27117323973654794 0.8806404698710601
[[391   4]
 [ 78 214]]
sens and spec is:
0.7328767123287672 0.9898734177215189
Epoch 1/1
687/687 [==============================] - 4s 6ms/step
0.21786302470993874 0.8995633236792757
[[329  66]
 [  3 289]]
sens and spec is:
0.9897260273972602 0.8329113924050633
Epoch 1/1
687/687 [==============================] - 4s 6ms/step
0.15922769281912014 0.9417758400089703
[[370  25]
 [ 15 277]]
sens and spec is:
0.9486301369863014 0.9367088607594937
517/517 [==============================] - 3s 5ms/step
sensitivity, specificity and a

## 2 layers cnn

In [16]:
model = Sequential()

model.add(Conv2D(filters = 10, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(50, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(10, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(1, activation = "softmax"))

model.compile(
 optimizer = "adam",
 loss = "binary_crossentropy",
 metrics = ["accuracy"]
)

model.fit(x=data_training, y=labels_training, batch_size=20, epochs=1,
     verbose=1, shuffle=True)
loss, acc = model.evaluate(x=data_sub_1, y=labels_sub_1, batch_size=20, verbose=1)

print(loss, acc)

Epoch 1/2
3438/3438 [==============================] - 39s 11ms/step - loss: 9.1629 - acc: 0.4252
Epoch 2/2
517/517 [==============================] - 3s 6ms/step
8.048283254154896 0.49516441051918486


In [10]:
sens_l = []
spec_l = []
acc_l = []

data_training = np.concatenate((data_sub_2, data_sub_3, data_sub_4, data_sub_5,
                                data_sub_6, data_sub_7, data_sub_8 , data_sub_9, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_2, labels_sub_3, labels_sub_4, labels_sub_5, 
                                  labels_sub_6, labels_sub_7, labels_sub_8, labels_sub_9, labels_sub_10))


def load_model():
    
    IMG_SIZE = 100
    model = Sequential()

    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(100, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation = "sigmoid"))

    model.compile(
     optimizer = "adam",
     loss = "binary_crossentropy",
     metrics = ["accuracy"]
    )
    return model

def train_evaluate(model, data_train, labels_train, data_test, labels_test):
    model.fit(x=data_train, y=labels_train, batch_size=5, epochs=1,
         verbose=1, shuffle=True)
    loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)
    print(loss, acc)
    y_pred = model.predict(data_test, verbose=0)
    y_pred = y_pred.round()
    y_pred = np.array([int(i) for i in y_pred])
    tn, fp, fn, tp = confusion_matrix(labels_test,y_pred).ravel()
    sens = tp/(tp+fn)
    sens_l.append(sens)
    spec = tn/(tn+fp)
    spec_l.append(spec)
    acc_l.append(acc)
    print(confusion_matrix(labels_test,y_pred))
    print("sens and spec is:")
    print(sens, spec)
    
from sklearn.model_selection import StratifiedKFold

if __name__ == "__main__":
    load_model()
    kFold = StratifiedKFold(n_splits=5, shuffle = True)
    for train, test in kFold.split(data_training,labels_training):
        model = None
        model = load_model()
        train_evaluate(model, data_training[train], labels_training[train], data_training[test], labels_training[test])

        
    loss, acc = model.evaluate(x=data_sub_1, y=labels_sub_1, batch_size=5, verbose=1)
    y_pred = model.predict(data_sub_1, verbose=0)
    y_pred = y_pred.round()
    y_pred = np.array([int(i) for i in y_pred])
    tn, fp, fn, tp = confusion_matrix(labels_sub_1,y_pred).ravel()
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)
    print("sensitivity, specificity and accuracy of models:")
    print(np.mean(sens_l))
    print(np.mean(spec_l))
    print(np.mean(acc_l))
    print(confusion_matrix(labels_sub_1,y_pred))
    print("sensitivity, specificity and accuracy of subject:")
    print(sens)
    print(spec)
    print(acc)

Epoch 1/1
689/689 [==============================] - 5s 7ms/step
0.35940811724681815 0.8519593675354569
[[376  20]
 [ 82 211]]
sens and spec is:
0.7201365187713311 0.9494949494949495
Epoch 1/1
688/688 [==============================] - 5s 7ms/step
0.3432843771649438 0.8808139589680142
[[352  43]
 [ 39 254]]
sens and spec is:
0.8668941979522184 0.8911392405063291
Epoch 1/1
687/687 [==============================] - 5s 7ms/step
0.4002058076315724 0.8180494976963365
[[339  56]
 [ 69 223]]
sens and spec is:
0.7636986301369864 0.8582278481012658
Epoch 1/1
687/687 [==============================] - 5s 7ms/step
0.3699795338975908 0.8471615800777724
[[315  80]
 [ 25 267]]
sens and spec is:
0.9143835616438356 0.7974683544303798
Epoch 1/1
687/687 [==============================] - 5s 7ms/step
0.42793708511794615 0.7787481878261899
[[254 141]
 [ 11 281]]
sens and spec is:
0.9623287671232876 0.6430379746835443
517/517 [==============================] - 3s 6ms/step
sensitivity, specificity and accu

In [11]:
sens_l = []
spec_l = []
acc_l = []

data_training = np.concatenate((data_sub_2, data_sub_3, data_sub_4, data_sub_5,
                                data_sub_6, data_sub_7, data_sub_8 , data_sub_9, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_2, labels_sub_3, labels_sub_4, labels_sub_5, 
                                  labels_sub_6, labels_sub_7, labels_sub_8, labels_sub_9, labels_sub_10))


def load_model():
    
    IMG_SIZE = 100
    model = Sequential()

    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(100, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation = "sigmoid"))

    model.compile(
     optimizer = "adam",
     loss = "binary_crossentropy",
     metrics = ["accuracy"]
    )
    return model

def train_evaluate(model, data_train, labels_train, data_test, labels_test):
    model.fit(x=data_train, y=labels_train, batch_size=5, epochs=1,
         verbose=1, shuffle=True)
    loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)
    print(loss, acc)
    y_pred = model.predict(data_test, verbose=0)
    y_pred = y_pred.round()
    y_pred = np.array([int(i) for i in y_pred])
    tn, fp, fn, tp = confusion_matrix(labels_test,y_pred).ravel()
    sens = tp/(tp+fn)
    sens_l.append(sens)
    spec = tn/(tn+fp)
    spec_l.append(spec)
    acc_l.append(acc)
    print(confusion_matrix(labels_test,y_pred))
    print("sens and spec is:")
    print(sens, spec)
    
from sklearn.model_selection import StratifiedKFold

if __name__ == "__main__":
    load_model()
    kFold = StratifiedKFold(n_splits=5, shuffle = True)
    for train, test in kFold.split(data_training,labels_training):
        model = None
        model = load_model()
        train_evaluate(model, data_training[train], labels_training[train], data_training[test], labels_training[test])

        
    loss, acc = model.evaluate(x=data_sub_1, y=labels_sub_1, batch_size=5, verbose=1)
    y_pred = model.predict(data_sub_1, verbose=0)
    y_pred = y_pred.round()
    y_pred = np.array([int(i) for i in y_pred])
    tn, fp, fn, tp = confusion_matrix(labels_sub_1,y_pred).ravel()
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)
    print("sensitivity, specificity and accuracy of models:")
    print(np.mean(sens_l))
    print(np.mean(spec_l))
    print(np.mean(acc_l))
    print(confusion_matrix(labels_sub_1,y_pred))
    print("sensitivity, specificity and accuracy of subject:")
    print(sens)
    print(spec)
    print(acc)

Epoch 1/1
689/689 [==============================] - 5s 8ms/step
0.5270439912450158 0.7024673518274623
[[194 202]
 [  3 290]]
sens and spec is:
0.9897610921501706 0.4898989898989899
Epoch 1/1
688/688 [==============================] - 5s 8ms/step
0.4465325037937958 0.7892441958795453
[[287 108]
 [ 37 256]]
sens and spec is:
0.8737201365187713 0.7265822784810126
Epoch 1/1
687/687 [==============================] - 5s 8ms/step
0.42435905454596273 0.7918486247243145
[[265 130]
 [ 13 279]]
sens and spec is:
0.9554794520547946 0.6708860759493671
Epoch 1/1
687/687 [==============================] - 6s 9ms/step
0.4674602402252141 0.7554585248925973
[[309  86]
 [ 82 210]]
sens and spec is:
0.7191780821917808 0.7822784810126582
Epoch 1/1
687/687 [==============================] - 6s 8ms/step
0.5293127962291848 0.7176128205080047
[[255 140]
 [ 54 238]]
sens and spec is:
0.815068493150685 0.6455696202531646
517/517 [==============================] - 3s 6ms/step
sensitivity, specificity and accura

In [12]:
sens_l = []
spec_l = []
acc_l = []

data_training = np.concatenate((data_sub_2, data_sub_3, data_sub_4, data_sub_5,
                                data_sub_6, data_sub_7, data_sub_8 , data_sub_9, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_2, labels_sub_3, labels_sub_4, labels_sub_5, 
                                  labels_sub_6, labels_sub_7, labels_sub_8, labels_sub_9, labels_sub_10))


def load_model():
    
    IMG_SIZE = 100
    model = Sequential()

    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(100, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation = "sigmoid"))

    model.compile(
     optimizer = "adam",
     loss = "binary_crossentropy",
     metrics = ["accuracy"]
    )
    return model

def train_evaluate(model, data_train, labels_train, data_test, labels_test):
    model.fit(x=data_train, y=labels_train, batch_size=5, epochs=1,
         verbose=1, shuffle=True)
    loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)
    print(loss, acc)
    y_pred = model.predict(data_test, verbose=0)
    y_pred = y_pred.round()
    y_pred = np.array([int(i) for i in y_pred])
    tn, fp, fn, tp = confusion_matrix(labels_test,y_pred).ravel()
    sens = tp/(tp+fn)
    sens_l.append(sens)
    spec = tn/(tn+fp)
    spec_l.append(spec)
    acc_l.append(acc)
    print(confusion_matrix(labels_test,y_pred))
    print("sens and spec is:")
    print(sens, spec)
    
from sklearn.model_selection import StratifiedKFold

if __name__ == "__main__":
    load_model()
    kFold = StratifiedKFold(n_splits=5, shuffle = True)
    for train, test in kFold.split(data_training,labels_training):
        model = None
        model = load_model()
        train_evaluate(model, data_training[train], labels_training[train], data_training[test], labels_training[test])

        
    loss, acc = model.evaluate(x=data_sub_1, y=labels_sub_1, batch_size=5, verbose=1)
    y_pred = model.predict(data_sub_1, verbose=0)
    y_pred = y_pred.round()
    y_pred = np.array([int(i) for i in y_pred])
    tn, fp, fn, tp = confusion_matrix(labels_sub_1,y_pred).ravel()
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)
    print("sensitivity, specificity and accuracy of models:")
    print(np.mean(sens_l))
    print(np.mean(spec_l))
    print(np.mean(acc_l))
    print(confusion_matrix(labels_sub_1,y_pred))
    print("sensitivity, specificity and accuracy of subject:")
    print(sens)
    print(spec)
    print(acc)

Epoch 1/1
689/689 [==============================] - 6s 8ms/step
0.49079634978048 0.7460087177888403
[[301  95]
 [ 80 213]]
sens and spec is:
0.726962457337884 0.76010101010101
Epoch 1/1
688/688 [==============================] - 6s 9ms/step
0.5000594366060267 0.7601744271598236
[[295 100]
 [ 65 228]]
sens and spec is:
0.7781569965870307 0.7468354430379747
Epoch 1/1
687/687 [==============================] - 6s 9ms/step
0.5190336027411742 0.7321688587705468
[[226 169]
 [ 15 277]]
sens and spec is:
0.9486301369863014 0.5721518987341773
Epoch 1/1
687/687 [==============================] - 6s 9ms/step
0.5364214138031613 0.7103347969844699
[[209 186]
 [ 13 279]]
sens and spec is:
0.9554794520547946 0.529113924050633
Epoch 1/1
687/687 [==============================] - 6s 9ms/step
0.5359583060396768 0.6943231535834169
[[212 183]
 [ 27 265]]
sens and spec is:
0.9075342465753424 0.5367088607594936
517/517 [==============================] - 3s 6ms/step
sensitivity, specificity and accuracy of 

## Subject 2 as test

In [2]:
sens_l = []
spec_l = []
acc_l = []

data_training = np.concatenate((data_sub_1, data_sub_3, data_sub_4, data_sub_5,
                                data_sub_6, data_sub_7, data_sub_8 , data_sub_9, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_1, labels_sub_3, labels_sub_4, labels_sub_5, 
                                  labels_sub_6, labels_sub_7, labels_sub_8, labels_sub_9, labels_sub_10))


def load_model():
    
    IMG_SIZE = 100
    model = Sequential()
    
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(100, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation = "sigmoid"))

    model.compile(
     optimizer = "adam",
     loss = "binary_crossentropy",
     metrics = ["accuracy"]
    )
    return model

def train_evaluate(model, data_train, labels_train, data_test, labels_test):
    model.fit(x=data_train, y=labels_train, batch_size=5, epochs=1,
         verbose=1, shuffle=True)
    loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)
    print(loss, acc)
    y_pred = model.predict(data_test, verbose=0)
    y_pred = y_pred.round()
    y_pred = np.array([int(i) for i in y_pred])
    tn, fp, fn, tp = confusion_matrix(labels_test,y_pred).ravel()
    sens = tp/(tp+fn)
    sens_l.append(sens)
    spec = tn/(tn+fp)
    spec_l.append(spec)
    acc_l.append(acc)
    print(confusion_matrix(labels_test,y_pred))
    print("sens and spec is:")
    print(sens, spec)
    
from sklearn.model_selection import StratifiedKFold

if __name__ == "__main__":
    load_model()
    kFold = StratifiedKFold(n_splits=5, shuffle = True)
    for train, test in kFold.split(data_training,labels_training):
        model = None
        model = load_model()
        train_evaluate(model, data_training[train], labels_training[train], data_training[test], labels_training[test])

        
    loss, acc = model.evaluate(x=data_sub_2, y=labels_sub_2, batch_size=5, verbose=1)
    y_pred = model.predict(data_sub_2, verbose=0)
    y_pred = y_pred.round()
    y_pred = np.array([int(i) for i in y_pred])
    tn, fp, fn, tp = confusion_matrix(labels_sub_2,y_pred).ravel()
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)
    print("sensitivity, specificity and accuracy of models:")
    print(np.mean(sens_l))
    print(np.mean(spec_l))
    print(np.mean(acc_l))
    print(confusion_matrix(labels_sub_2,y_pred))
    print("sensitivity, specificity and accuracy of subject:")
    print(sens)
    print(spec)
    print(acc)

Epoch 1/1
720/720 [==============================] - 3s 4ms/step
0.2466100368506482 0.9152777812754115
[[402  10]
 [ 51 257]]
sens and spec is:
0.8344155844155844 0.9757281553398058
Epoch 1/1
720/720 [==============================] - 3s 4ms/step
0.16673402197759263 0.9319444482939111
[[381  31]
 [ 18 290]]
sens and spec is:
0.9415584415584416 0.9247572815533981
Epoch 1/1
720/720 [==============================] - 3s 5ms/step
0.29524221702983294 0.8791666720062494
[[329  83]
 [  4 304]]
sens and spec is:
0.987012987012987 0.7985436893203883
Epoch 1/1
719/719 [==============================] - 3s 5ms/step
0.2068038180642742 0.9221140500650287
[[404   7]
 [ 49 259]]
sens and spec is:
0.8409090909090909 0.9829683698296837
Epoch 1/1
718/718 [==============================] - 3s 5ms/step
0.22927629049415998 0.9011142103618898
[[397  14]
 [ 57 250]]
sens and spec is:
0.8143322475570033 0.9659367396593674
358/358 [==============================] - 2s 4ms/step
sensitivity, specificity and accu

In [3]:
sens_l = []
spec_l = []
acc_l = []

data_training = np.concatenate((data_sub_1, data_sub_3, data_sub_4, data_sub_5,
                                data_sub_6, data_sub_7, data_sub_8 , data_sub_9, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_1, labels_sub_3, labels_sub_4, labels_sub_5, 
                                  labels_sub_6, labels_sub_7, labels_sub_8, labels_sub_9, labels_sub_10))


def load_model():
    
    IMG_SIZE = 100
    model = Sequential()
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(100, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation = "sigmoid"))

    model.compile(
     optimizer = "adam",
     loss = "binary_crossentropy",
     metrics = ["accuracy"]
    )
    return model

def train_evaluate(model, data_train, labels_train, data_test, labels_test):
    model.fit(x=data_train, y=labels_train, batch_size=5, epochs=1,
         verbose=1, shuffle=True)
    loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)
    print(loss, acc)
    y_pred = model.predict(data_test, verbose=0)
    y_pred = y_pred.round()
    y_pred = np.array([int(i) for i in y_pred])
    tn, fp, fn, tp = confusion_matrix(labels_test,y_pred).ravel()
    sens = tp/(tp+fn)
    sens_l.append(sens)
    spec = tn/(tn+fp)
    spec_l.append(spec)
    acc_l.append(acc)
    print(confusion_matrix(labels_test,y_pred))
    print("sens and spec is:")
    print(sens, spec)
    
from sklearn.model_selection import StratifiedKFold

if __name__ == "__main__":
    load_model()
    kFold = StratifiedKFold(n_splits=5, shuffle = True)
    for train, test in kFold.split(data_training,labels_training):
        model = None
        model = load_model()
        train_evaluate(model, data_training[train], labels_training[train], data_training[test], labels_training[test])

        
    loss, acc = model.evaluate(x=data_sub_2, y=labels_sub_2, batch_size=5, verbose=1)
    y_pred = model.predict(data_sub_2, verbose=0)
    y_pred = y_pred.round()
    y_pred = np.array([int(i) for i in y_pred])
    tn, fp, fn, tp = confusion_matrix(labels_sub_2,y_pred).ravel()
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)
    print("sensitivity, specificity and accuracy of models:")
    print(np.mean(sens_l))
    print(np.mean(spec_l))
    print(np.mean(acc_l))
    print(confusion_matrix(labels_sub_2,y_pred))
    print("sensitivity, specificity and accuracy of subject:")
    print(sens)
    print(spec)
    print(acc)

Epoch 1/1
720/720 [==============================] - 4s 6ms/step
0.3499336114424902 0.8305555632751849
[[358  54]
 [ 68 240]]
sens and spec is:
0.7792207792207793 0.8689320388349514
Epoch 1/1
720/720 [==============================] - 4s 6ms/step
0.3441493534976164 0.8555555621989899
[[335  77]
 [ 27 281]]
sens and spec is:
0.9123376623376623 0.8131067961165048
Epoch 1/1
720/720 [==============================] - 4s 6ms/step
0.4362986243924954 0.8013888978295856
[[372  40]
 [103 205]]
sens and spec is:
0.6655844155844156 0.9029126213592233
Epoch 1/1
719/719 [==============================] - 4s 6ms/step
0.4496108311926622 0.7538247663263485
[[260 151]
 [ 26 282]]
sens and spec is:
0.9155844155844156 0.6326034063260341
Epoch 1/1
718/718 [==============================] - 4s 6ms/step
0.3786539401477047 0.8300835725573776
[[354  57]
 [ 65 242]]
sens and spec is:
0.7882736156351792 0.8613138686131386
358/358 [==============================] - 2s 6ms/step
sensitivity, specificity and accura

In [4]:
sens_l = []
spec_l = []
acc_l = []

data_training = np.concatenate((data_sub_1, data_sub_3, data_sub_4, data_sub_5,
                                data_sub_6, data_sub_7, data_sub_8 , data_sub_9, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_1, labels_sub_3, labels_sub_4, labels_sub_5, 
                                  labels_sub_6, labels_sub_7, labels_sub_8, labels_sub_9, labels_sub_10))


def load_model():
    
    IMG_SIZE = 100
    model = Sequential()
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(100, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation = "sigmoid"))

    model.compile(
     optimizer = "adam",
     loss = "binary_crossentropy",
     metrics = ["accuracy"]
    )
    return model

def train_evaluate(model, data_train, labels_train, data_test, labels_test):
    model.fit(x=data_train, y=labels_train, batch_size=5, epochs=1,
         verbose=1, shuffle=True)
    loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)
    print(loss, acc)
    y_pred = model.predict(data_test, verbose=0)
    y_pred = y_pred.round()
    y_pred = np.array([int(i) for i in y_pred])
    tn, fp, fn, tp = confusion_matrix(labels_test,y_pred).ravel()
    sens = tp/(tp+fn)
    sens_l.append(sens)
    spec = tn/(tn+fp)
    spec_l.append(spec)
    acc_l.append(acc)
    print(confusion_matrix(labels_test,y_pred))
    print("sens and spec is:")
    print(sens, spec)
    
from sklearn.model_selection import StratifiedKFold

if __name__ == "__main__":
    load_model()
    kFold = StratifiedKFold(n_splits=5, shuffle = True)
    for train, test in kFold.split(data_training,labels_training):
        model = None
        model = load_model()
        train_evaluate(model, data_training[train], labels_training[train], data_training[test], labels_training[test])

        
    loss, acc = model.evaluate(x=data_sub_2, y=labels_sub_2, batch_size=5, verbose=1)
    y_pred = model.predict(data_sub_2, verbose=0)
    y_pred = y_pred.round()
    y_pred = np.array([int(i) for i in y_pred])
    tn, fp, fn, tp = confusion_matrix(labels_sub_2,y_pred).ravel()
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)
    print("sensitivity, specificity and accuracy of models:")
    print(np.mean(sens_l))
    print(np.mean(spec_l))
    print(np.mean(acc_l))
    print(confusion_matrix(labels_sub_2,y_pred))
    print("sensitivity, specificity and accuracy of subject:")
    print(sens)
    print(spec)
    print(acc)

Epoch 1/1
720/720 [==============================] - 5s 6ms/step
0.5179859543374429 0.7125000074091885
[[361  51]
 [156 152]]
sens and spec is:
0.4935064935064935 0.8762135922330098
Epoch 1/1
720/720 [==============================] - 5s 7ms/step
0.40687322154796374 0.8166666746967368
[[284 128]
 [  4 304]]
sens and spec is:
0.987012987012987 0.6893203883495146
Epoch 1/1
720/720 [==============================] - 5s 7ms/step
0.5085385063171594 0.7250000111137828
[[327  85]
 [113 195]]
sens and spec is:
0.6331168831168831 0.7936893203883495
Epoch 1/1
719/719 [==============================] - 5s 7ms/step
0.5151722642731683 0.7677329704890828
[[363  48]
 [119 189]]
sens and spec is:
0.6136363636363636 0.8832116788321168
Epoch 1/1
718/718 [==============================] - 5s 7ms/step
0.49965518850864893 0.7130919290410775
[[380  31]
 [175 132]]
sens and spec is:
0.42996742671009774 0.9245742092457421
358/358 [==============================] - 2s 6ms/step
sensitivity, specificity and accu

In [5]:
sens_l = []
spec_l = []
acc_l = []

data_training = np.concatenate((data_sub_1, data_sub_3, data_sub_4, data_sub_5,
                                data_sub_6, data_sub_7, data_sub_8 , data_sub_9, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_1, labels_sub_3, labels_sub_4, labels_sub_5, 
                                  labels_sub_6, labels_sub_7, labels_sub_8, labels_sub_9, labels_sub_10))


def load_model():
    
    IMG_SIZE = 100
    model = Sequential()
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(100, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation = "sigmoid"))

    model.compile(
     optimizer = "adam",
     loss = "binary_crossentropy",
     metrics = ["accuracy"]
    )
    return model

def train_evaluate(model, data_train, labels_train, data_test, labels_test):
    model.fit(x=data_train, y=labels_train, batch_size=5, epochs=1,
         verbose=1, shuffle=True)
    loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)
    print(loss, acc)
    y_pred = model.predict(data_test, verbose=0)
    y_pred = y_pred.round()
    y_pred = np.array([int(i) for i in y_pred])
    tn, fp, fn, tp = confusion_matrix(labels_test,y_pred).ravel()
    sens = tp/(tp+fn)
    sens_l.append(sens)
    spec = tn/(tn+fp)
    spec_l.append(spec)
    acc_l.append(acc)
    print(confusion_matrix(labels_test,y_pred))
    print("sens and spec is:")
    print(sens, spec)
    
from sklearn.model_selection import StratifiedKFold

if __name__ == "__main__":
    load_model()
    kFold = StratifiedKFold(n_splits=5, shuffle = True)
    for train, test in kFold.split(data_training,labels_training):
        model = None
        model = load_model()
        train_evaluate(model, data_training[train], labels_training[train], data_training[test], labels_training[test])

        
    loss, acc = model.evaluate(x=data_sub_2, y=labels_sub_2, batch_size=5, verbose=1)
    y_pred = model.predict(data_sub_2, verbose=0)
    y_pred = y_pred.round()
    y_pred = np.array([int(i) for i in y_pred])
    tn, fp, fn, tp = confusion_matrix(labels_sub_2,y_pred).ravel()
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)
    print("sensitivity, specificity and accuracy of models:")
    print(np.mean(sens_l))
    print(np.mean(spec_l))
    print(np.mean(acc_l))
    print(confusion_matrix(labels_sub_2,y_pred))
    print("sensitivity, specificity and accuracy of subject:")
    print(sens)
    print(spec)
    print(acc)

Epoch 1/1
720/720 [==============================] - 5s 7ms/step
0.5750315971672535 0.6500000077196293
[[183 229]
 [ 23 285]]
sens and spec is:
0.9253246753246753 0.4441747572815534
Epoch 1/1
720/720 [==============================] - 5s 7ms/step
0.630590307008889 0.5944444488527046
[[120 292]
 [  0 308]]
sens and spec is:
1.0 0.2912621359223301
Epoch 1/1
720/720 [==============================] - 5s 7ms/step
0.543579533799655 0.6666666762903333
[[232 180]
 [ 60 248]]
sens and spec is:
0.8051948051948052 0.5631067961165048
Epoch 1/1
719/719 [==============================] - 5s 8ms/step
0.510307967704261 0.7649513308751567
[[317  94]
 [ 75 233]]
sens and spec is:
0.7564935064935064 0.7712895377128953
Epoch 1/1
718/718 [==============================] - 6s 8ms/step
0.5570936507563571 0.6559888642270917
[[369  42]
 [205 102]]
sens and spec is:
0.3322475570032573 0.8978102189781022
358/358 [==============================] - 2s 6ms/step
sensitivity, specificity and accuracy of models:
0.76

## 2 Layers - subject 2 as test

In [20]:
data_training = np.concatenate((data_sub_1, data_sub_3, data_sub_4, data_sub_5,
                                data_sub_6, data_sub_7, data_sub_8 , data_sub_9, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_1, labels_sub_3, labels_sub_4, labels_sub_5, 
                                  labels_sub_6, labels_sub_7, labels_sub_8, labels_sub_9, labels_sub_10))


IMG_SIZE = 100
model = Sequential()

model.add(Conv2D(filters = 30, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 15, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(100, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(50, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(1, activation = "softmax"))

model.compile(
 optimizer = "adam",
 loss = "binary_crossentropy",
 metrics = ["accuracy"]
)

model.fit(x=data_training, y=labels_training, batch_size=20, epochs=2,
     verbose=1, shuffle=True)
loss, acc = model.evaluate(x=data_sub_2, y=labels_sub_2, batch_size=20, verbose=1)

print(loss, acc)

Epoch 1/2
3597/3597 [==============================] - 132s 37ms/step - loss: 9.1213 - acc: 0.4279
Epoch 2/2
358/358 [==============================] - 5s 15ms/step
7.971192417864046 0.49999999933402633


In [15]:
data_training = np.concatenate((data_sub_1, data_sub_3, data_sub_4, data_sub_5,
                                data_sub_6, data_sub_7, data_sub_8 , data_sub_9, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_1, labels_sub_3, labels_sub_4, labels_sub_5, 
                                  labels_sub_6, labels_sub_7, labels_sub_8, labels_sub_9, labels_sub_10), axis = 0)



def load_model():
    
    IMG_SIZE = 100
    model = Sequential()
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(100, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation = "sigmoid"))

    model.compile(
     optimizer = "adam",
     loss = "binary_crossentropy",
     metrics = ["accuracy"]
    )
    return model

def train_evaluate(model, data_train, labels_train, data_test, labels_test):
    model.fit(x=data_train, y=labels_train, batch_size=5, epochs=1,
         verbose=1, shuffle=True)
    loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)
    print(loss, acc)

from sklearn.model_selection import StratifiedKFold

if __name__ == "__main__":
    load_model()
    kFold = StratifiedKFold(n_splits=5, shuffle = True)
    for train, test in kFold.split(data_training,labels_training):
        model = None
        model = load_model()
        train_evaluate(model, data_training[train], labels_training[train], data_training[test], labels_training[test])

        
    loss, acc = model.evaluate(x=data_sub_2, y=labels_sub_2, batch_size=5, verbose=1)

    print(loss, acc)

Epoch 1/1
720/720 [==============================] - 5s 6ms/step
0.23498108607681287 0.9194444473832846
Epoch 1/1
720/720 [==============================] - 5s 7ms/step
0.3098798149989711 0.8736111164713899
Epoch 1/1
720/720 [==============================] - 5s 7ms/step
0.19984528016518904 0.9194444492459297
Epoch 1/1
719/719 [==============================] - 5s 7ms/step
0.23456822238054245 0.8901251790545083
Epoch 1/1
718/718 [==============================] - 5s 7ms/step
0.2427278352955818 0.8983286960377335
358/358 [==============================] - 2s 4ms/step
0.36065692262588767 0.8463687229922364


In [18]:
data_training = np.concatenate((data_sub_1, data_sub_3, data_sub_4, data_sub_5,
                                data_sub_6, data_sub_7, data_sub_8 , data_sub_9, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_1, labels_sub_3, labels_sub_4, labels_sub_5, 
                                  labels_sub_6, labels_sub_7, labels_sub_8, labels_sub_9, labels_sub_10), axis = 0)



def load_model():
    
    IMG_SIZE = 100
    model = Sequential()
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(100, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation = "sigmoid"))

    model.compile(
     optimizer = "adam",
     loss = "binary_crossentropy",
     metrics = ["accuracy"]
    )
    return model

def train_evaluate(model, data_train, labels_train, data_test, labels_test):
    model.fit(x=data_train, y=labels_train, batch_size=5, epochs=1,
         verbose=1, shuffle=True)
    loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)
    print(loss, acc)

from sklearn.model_selection import StratifiedKFold

if __name__ == "__main__":
    load_model()
    kFold = StratifiedKFold(n_splits=5, shuffle = True)
    for train, test in kFold.split(data_training,labels_training):
        model = None
        model = load_model()
        train_evaluate(model, data_training[train], labels_training[train], data_training[test], labels_training[test])

        
    loss, acc = model.evaluate(x=data_sub_2, y=labels_sub_2, batch_size=10, verbose=1)

    print(loss, acc)

Epoch 1/1
720/720 [==============================] - 6s 9ms/step
0.42704053688794374 0.8222222296107147
Epoch 1/1
720/720 [==============================] - 6s 9ms/step
0.4347013759478513 0.7833333417980207
Epoch 1/1
720/720 [==============================] - 7s 9ms/step
0.45845804860841277 0.7472222277687656
Epoch 1/1
719/719 [==============================] - 7s 9ms/step
0.3964750896258505 0.7844228174780937
Epoch 1/1
718/718 [==============================] - 7s 9ms/step
0.38233953352023986 0.7896936025086552
358/358 [==============================] - 2s 5ms/step
0.470865279852345 0.6955307282549043


In [17]:
data_training = np.concatenate((data_sub_1, data_sub_3, data_sub_4, data_sub_5,
                                data_sub_6, data_sub_7, data_sub_8 , data_sub_9, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_1, labels_sub_3, labels_sub_4, labels_sub_5, 
                                  labels_sub_6, labels_sub_7, labels_sub_8, labels_sub_9, labels_sub_10), axis = 0)



def load_model():
    
    IMG_SIZE = 100
    model = Sequential()
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(100, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation = "sigmoid"))

    model.compile(
     optimizer = "adam",
     loss = "binary_crossentropy",
     metrics = ["accuracy"]
    )
    return model

def train_evaluate(model, data_train, labels_train, data_test, labels_test):
    model.fit(x=data_train, y=labels_train, batch_size=5, epochs=1,
         verbose=1, shuffle=True)
    loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)
    print(loss, acc)

from sklearn.model_selection import StratifiedKFold

if __name__ == "__main__":
    load_model()
    kFold = StratifiedKFold(n_splits=5, shuffle = True)
    for train, test in kFold.split(data_training,labels_training):
        model = None
        model = load_model()
        train_evaluate(model, data_training[train], labels_training[train], data_training[test], labels_training[test])

        
    loss, acc = model.evaluate(x=data_sub_2, y=labels_sub_2, batch_size=5, verbose=1)

    print(loss, acc)

Epoch 1/1
720/720 [==============================] - 6s 8ms/step
0.5645190679157773 0.6361111137602065
Epoch 1/1
720/720 [==============================] - 7s 10ms/step
0.5486681829206645 0.6805555655931433
Epoch 1/1
720/720 [==============================] - 6s 9ms/step
0.527298195546286 0.651388892903924
Epoch 1/1
719/719 [==============================] - 6s 9ms/step
0.4594330576757521 0.7538247669688186
Epoch 1/1
718/718 [==============================] - 6s 9ms/step
0.4895921841881194 0.7075209014304501
358/358 [==============================] - 2s 6ms/step
0.453517316132992 0.8212290601024415


## Subject 3 as test

In [19]:
data_training = np.concatenate((data_sub_2, data_sub_1, data_sub_4, data_sub_5,
                                data_sub_6, data_sub_7, data_sub_8 , data_sub_9, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_2, labels_sub_1, labels_sub_4, labels_sub_5, 
                                  labels_sub_6, labels_sub_7, labels_sub_8, labels_sub_9, labels_sub_10))

  
IMG_SIZE = 100
model = Sequential()



model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(100, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(50, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(1, activation = "softmax"))

model.compile(
 optimizer = "adam",
 loss = "binary_crossentropy",
 metrics = ["accuracy"]
)

model.fit(x=data_training, y=labels_training, batch_size=5, epochs=1,
     verbose=1, shuffle=True)
loss, acc = model.evaluate(x=data_sub_3, y=labels_sub_3, batch_size=5, verbose=1)

print(loss, acc)

Epoch 1/1
453/453 [==============================] - 5s 11ms/step
8.023981778826956 0.4966887417218543


In [21]:
data_training = np.concatenate((data_sub_2, data_sub_1, data_sub_4, data_sub_5,
                                data_sub_6, data_sub_7, data_sub_8 , data_sub_9, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_2, labels_sub_1, labels_sub_4, labels_sub_5, 
                                  labels_sub_6, labels_sub_7, labels_sub_8, labels_sub_9, labels_sub_10))


def load_model():
    
    IMG_SIZE = 100
    model = Sequential()
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(100, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation = "sigmoid"))

    model.compile(
     optimizer = "adam",
     loss = "binary_crossentropy",
     metrics = ["accuracy"]
    )
    return model

def train_evaluate(model, data_train, labels_train, data_test, labels_test):
    model.fit(x=data_train, y=labels_train, batch_size=5, epochs=1,
         verbose=1, shuffle=True)
    loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)
    print(loss, acc)

from sklearn.model_selection import StratifiedKFold

if __name__ == "__main__":
    load_model()
    kFold = StratifiedKFold(n_splits=5, shuffle = True)
    for train, test in kFold.split(data_training,labels_training):
        model = None
        model = load_model()
        train_evaluate(model, data_training[train], labels_training[train], data_training[test], labels_training[test])

        
    loss, acc = model.evaluate(x=data_sub_3, y=labels_sub_3, batch_size=5, verbose=1)

    print(loss, acc)

Epoch 1/1
701/701 [==============================] - 7s 9ms/step
0.16043702084086198 0.9229671937890468
Epoch 1/1
701/701 [==============================] - 7s 10ms/step
0.22844308203274913 0.9358059950544899
Epoch 1/1
701/701 [==============================] - 7s 10ms/step
0.18650562736835496 0.9300998598124604
Epoch 1/1
700/700 [==============================] - 7s 10ms/step
0.27219909891220073 0.8614285778786455
Epoch 1/1
699/699 [==============================] - 7s 10ms/step
0.20069729632701996 0.9113018648733567
453/453 [==============================] - 2s 5ms/step
0.9468950280512602 0.5871964749976762


## 2 layers and subject 3

In [50]:
data_training = np.concatenate((data_sub_2, data_sub_1, data_sub_4, data_sub_5,
                                data_sub_6, data_sub_7, data_sub_8 , data_sub_9, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_2, labels_sub_1, labels_sub_4, labels_sub_5, 
                                  labels_sub_6, labels_sub_7, labels_sub_8, labels_sub_9, labels_sub_10))

limit = math.ceil(len(data_training)/len(data_sub_3))

for i in range(limit):
    data_test = np.concatenate((data_sub_3, data_sub_3)) 
    labels_test = np.concatenate((labels_sub_3, labels_sub_3))
    
IMG_SIZE = 100
model = Sequential()



model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(100, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(50, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(1, activation = "softmax"))

model.compile(
 optimizer = "adam",
 loss = "binary_crossentropy",
 metrics = ["accuracy"]
)

model.fit(x=data_training, y=labels_training, batch_size=5, epochs=3,
     verbose=1, shuffle=True)
loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)

print(loss, acc)

Epoch 1/3
4183/4183 [==============================] - 46s 11ms/step - loss: 9.0288 - acc: 0.4337
Epoch 2/3
4183/4183 [==============================] - 44s 10ms/step - loss: 9.0288 - acc: 0.4337
Epoch 3/3
906/906 [==============================] - 5s 6ms/step
8.023981777116436 0.4966887418863263


In [23]:
data_training = np.concatenate((data_sub_2, data_sub_1, data_sub_4, data_sub_5,
                                data_sub_6, data_sub_7, data_sub_8 , data_sub_9, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_2, labels_sub_1, labels_sub_4, labels_sub_5, 
                                  labels_sub_6, labels_sub_7, labels_sub_8, labels_sub_9, labels_sub_10))


def load_model():
    
    IMG_SIZE = 100
    model = Sequential()
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(100, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation = "sigmoid"))

    model.compile(
     optimizer = "adam",
     loss = "binary_crossentropy",
     metrics = ["accuracy"]
    )
    return model

def train_evaluate(model, data_train, labels_train, data_test, labels_test):
    model.fit(x=data_train, y=labels_train, batch_size=5, epochs=1,
         verbose=1, shuffle=True)
    loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)
    print(loss, acc)

from sklearn.model_selection import StratifiedKFold

if __name__ == "__main__":
    load_model()
    kFold = StratifiedKFold(n_splits=5, shuffle = True)
    for train, test in kFold.split(data_training,labels_training):
        model = None
        model = load_model()
        train_evaluate(model, data_training[train], labels_training[train], data_training[test], labels_training[test])

        
    loss, acc = model.evaluate(x=data_sub_3, y=labels_sub_3, batch_size=5, verbose=1)

    print(loss, acc)

Epoch 1/1
701/701 [==============================] - 8s 11ms/step
0.44941455995751683 0.8687589215313997
Epoch 1/1
701/701 [==============================] - 8s 12ms/step
0.4438470402883948 0.8031383828285247
Epoch 1/1
701/701 [==============================] - 8s 12ms/step
0.4653443654468708 0.748930111654985
Epoch 1/1
700/700 [==============================] - 8s 12ms/step
0.5021579936080213 0.6700000034911292
Epoch 1/1
699/699 [==============================] - 9s 12ms/step
0.43321258897547715 0.7839771181302351
453/453 [==============================] - 3s 6ms/step
0.7692995113387465 0.5584988998656242


In [24]:
data_training = np.concatenate((data_sub_2, data_sub_1, data_sub_4, data_sub_5,
                                data_sub_6, data_sub_7, data_sub_8 , data_sub_9, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_2, labels_sub_1, labels_sub_4, labels_sub_5, 
                                  labels_sub_6, labels_sub_7, labels_sub_8, labels_sub_9, labels_sub_10))


def load_model():
    
    IMG_SIZE = 100
    model = Sequential()
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(100, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation = "sigmoid"))

    model.compile(
     optimizer = "adam",
     loss = "binary_crossentropy",
     metrics = ["accuracy"]
    )
    return model

def train_evaluate(model, data_train, labels_train, data_test, labels_test):
    model.fit(x=data_train, y=labels_train, batch_size=5, epochs=1,
         verbose=1, shuffle=True)
    loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)
    print(loss, acc)

from sklearn.model_selection import StratifiedKFold

if __name__ == "__main__":
    load_model()
    kFold = StratifiedKFold(n_splits=5, shuffle = True)
    for train, test in kFold.split(data_training,labels_training):
        model = None
        model = load_model()
        train_evaluate(model, data_training[train], labels_training[train], data_training[test], labels_training[test])

        
    loss, acc = model.evaluate(x=data_sub_3, y=labels_sub_3, batch_size=5, verbose=1)

    print(loss, acc)

Epoch 1/1
701/701 [==============================] - 9s 13ms/step
0.5326136752495412 0.7389443751408609
Epoch 1/1
701/701 [==============================] - 9s 13ms/step
0.5448640272042551 0.6376604902506214
Epoch 1/1
701/701 [==============================] - 9s 13ms/step
0.5491505335756018 0.6205420875005137
Epoch 1/1
700/700 [==============================] - 9s 13ms/step
0.574284538520234 0.6828571518617017
Epoch 1/1
699/699 [==============================] - 9s 13ms/step
0.5682353702225911 0.6952789801256851
453/453 [==============================] - 3s 6ms/step
0.6017461236870578 0.596026496710893


In [25]:
data_training = np.concatenate((data_sub_2, data_sub_1, data_sub_4, data_sub_5,
                                data_sub_6, data_sub_7, data_sub_8 , data_sub_9, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_2, labels_sub_1, labels_sub_4, labels_sub_5, 
                                  labels_sub_6, labels_sub_7, labels_sub_8, labels_sub_9, labels_sub_10))


def load_model():
    
    IMG_SIZE = 100
    model = Sequential()
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(100, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation = "sigmoid"))

    model.compile(
     optimizer = "adam",
     loss = "binary_crossentropy",
     metrics = ["accuracy"]
    )
    return model

def train_evaluate(model, data_train, labels_train, data_test, labels_test):
    model.fit(x=data_train, y=labels_train, batch_size=5, epochs=1,
         verbose=1, shuffle=True)
    loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)
    print(loss, acc)

from sklearn.model_selection import StratifiedKFold

if __name__ == "__main__":
    load_model()
    kFold = StratifiedKFold(n_splits=5, shuffle = True)
    for train, test in kFold.split(data_training,labels_training):
        model = None
        model = load_model()
        train_evaluate(model, data_training[train], labels_training[train], data_training[test], labels_training[test])

        
    loss, acc = model.evaluate(x=data_sub_3, y=labels_sub_3, batch_size=5, verbose=1)

    print(loss, acc)

Epoch 1/1
701/701 [==============================] - 10s 14ms/step
0.6834178019352204 0.5734664776525892
Epoch 1/1
701/701 [==============================] - 10s 14ms/step
0.6585237723529423 0.573466477227449
Epoch 1/1
701/701 [==============================] - 10s 14ms/step
0.6775578774841298 0.5734664773337341
Epoch 1/1
700/700 [==============================] - 11s 16ms/step
0.6679660648107528 0.5714285723865032
Epoch 1/1
699/699 [==============================] - 10s 15ms/step
0.6828562187534545 0.5736766819747562
453/453 [==============================] - 3s 6ms/step
0.7075360919992392 0.5033112582781457


## Subject 4 

In [64]:
data_training = np.concatenate((data_sub_2, data_sub_1, data_sub_3, data_sub_5,
                                data_sub_6, data_sub_7, data_sub_8 , data_sub_9, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_2, labels_sub_1, labels_sub_3, labels_sub_5, 
                                  labels_sub_6, labels_sub_7, labels_sub_8, labels_sub_9, labels_sub_10))

limit = math.ceil(len(data_training)/len(data_sub_4))

for i in range(limit):
    data_test = np.array([data_sub_4,], limit)
    labels_test = np.array([labels_sub_4,], limit)
    
IMG_SIZE = 100
model = Sequential()



model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(100, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(50, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(1, activation = "softmax"))

model.compile(
 optimizer = "adam",
 loss = "binary_crossentropy",
 metrics = ["accuracy"]
)

model.fit(x=data_training, y=labels_training, batch_size=5, epochs=3,
     verbose=1, shuffle=True)
loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)

print(loss, acc)

Epoch 1/3
4334/4334 [==============================] - 49s 11ms/step - loss: 8.4420 - acc: 0.4705
Epoch 2/3
4334/4334 [==============================] - 47s 11ms/step - loss: 8.4420 - acc: 0.4705
Epoch 3/3
604/604 [==============================] - 4s 7ms/step
15.942384719848633 0.0


In [26]:
data_training = np.concatenate((data_sub_2, data_sub_1, data_sub_3, data_sub_5,
                                data_sub_6, data_sub_7, data_sub_8 , data_sub_9, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_2, labels_sub_1, labels_sub_3, labels_sub_5, 
                                  labels_sub_6, labels_sub_7, labels_sub_8, labels_sub_9, labels_sub_10))


def load_model():
    
    IMG_SIZE = 100
    model = Sequential()
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(100, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation = "sigmoid"))

    model.compile(
     optimizer = "adam",
     loss = "binary_crossentropy",
     metrics = ["accuracy"]
    )
    return model

def train_evaluate(model, data_train, labels_train, data_test, labels_test):
    model.fit(x=data_train, y=labels_train, batch_size=5, epochs=1,
         verbose=1, shuffle=True)
    loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)
    print(loss, acc)

from sklearn.model_selection import StratifiedKFold

if __name__ == "__main__":
    load_model()
    kFold = StratifiedKFold(n_splits=5, shuffle = True)
    for train, test in kFold.split(data_training,labels_training):
        model = None
        model = load_model()
        train_evaluate(model, data_training[train], labels_training[train], data_training[test], labels_training[test])

        
    loss, acc = model.evaluate(x=data_sub_4, y=labels_sub_4, batch_size=5, verbose=1)
    print(loss, acc)
    y_pred = model.predict(data_sub_4, verbose=0)
    y_pred = np.array([int(round(i[0])) for i in y_pred])

    from sklearn.metrics import confusion_matrix
    confusion_matrix(labels_sub_4,y_pred)

Epoch 1/1
731/731 [==============================] - 10s 13ms/step
0.22487623746176894 0.9110807162466192
Epoch 1/1
731/731 [==============================] - 10s 14ms/step
0.22624518324665346 0.9042407711700278
Epoch 1/1
731/731 [==============================] - 10s 14ms/step
0.28448650558070165 0.8837209348190989
Epoch 1/1
730/730 [==============================] - 10s 14ms/step
0.2699483315355132 0.8931506895651556
Epoch 1/1
730/730 [==============================] - 11s 14ms/step
0.19516684742329868 0.9219178126691139
302/302 [==============================] - 2s 5ms/step
0.3111865697120199 0.8708609343561905


In [66]:
data_training = np.concatenate((data_sub_2, data_sub_1, data_sub_3, data_sub_5,
                                data_sub_6, data_sub_7, data_sub_8 , data_sub_9, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_2, labels_sub_1, labels_sub_3, labels_sub_5, 
                                  labels_sub_6, labels_sub_7, labels_sub_8, labels_sub_9, labels_sub_10))

limit = math.ceil(len(data_training)/len(data_sub_4))

for i in range(limit):
    data_test = np.concatenate((data_sub_4, data_sub_4)) 
    labels_test = np.concatenate((labels_sub_4, labels_sub_4))
    
IMG_SIZE = 100
model = Sequential()



model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(100, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(50, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(1, activation = "softmax"))

model.compile(
 optimizer = "adam",
 loss = "binary_crossentropy",
 metrics = ["accuracy"]
)

model.fit(x=data_training, y=labels_training, batch_size=5, epochs=3,
     verbose=1, shuffle=True)
loss, acc = model.evaluate(x=data_sub_4, y=labels_sub_4, batch_size=5, verbose=1)

print(loss, acc)

Epoch 1/3
4334/4334 [==============================] - 50s 11ms/step - loss: 8.4420 - acc: 0.4705
Epoch 2/3
4334/4334 [==============================] - 48s 11ms/step - loss: 8.4420 - acc: 0.4705
Epoch 3/3
604/604 [==============================] - 4s 7ms/step
15.942384719848633 0.0


In [57]:
data_training = np.concatenate((data_sub_2, data_sub_1, data_sub_3, data_sub_5,
                                data_sub_6, data_sub_7, data_sub_8 , data_sub_9, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_2, labels_sub_1, labels_sub_3, labels_sub_5, 
                                  labels_sub_6, labels_sub_7, labels_sub_8, labels_sub_9, labels_sub_10))


def load_model():
    
    IMG_SIZE = 100
    model = Sequential()
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(100, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation = "sigmoid"))

    model.compile(
     optimizer = "adam",
     loss = "binary_crossentropy",
     metrics = ["accuracy"]
    )
    return model

def train_evaluate(model, data_train, labels_train, data_test, labels_test):
    model.fit(x=data_train, y=labels_train, batch_size=5, epochs=1,
         verbose=1, shuffle=True)
    loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)
    print(loss, acc)

from sklearn.model_selection import StratifiedKFold

if __name__ == "__main__":
    load_model()
    kFold = StratifiedKFold(n_splits=5, shuffle = True)
    for train, test in kFold.split(data_training,labels_training):
        model = None
        model = load_model()
        train_evaluate(model, data_training[train], labels_training[train], data_training[test], labels_training[test])

        
    loss, acc = model.evaluate(x=data_sub_4, y=labels_sub_4, batch_size=5, verbose=1)
    print(loss, acc)
    y_pred = model.predict(data_sub_4, verbose=0)
    y_pred = np.array([int(round(i[0])) for i in y_pred])

    from sklearn.metrics import confusion_matrix
    confusion_matrix(labels_sub_4,y_pred)

Epoch 1/1
731/731 [==============================] - 11s 15ms/step
0.5235446862692954 0.7346101318028655
Epoch 1/1
731/731 [==============================] - 11s 15ms/step
0.47563557553827435 0.7619699114161542
Epoch 1/1
731/731 [==============================] - 12s 16ms/step
0.521079785785499 0.7647058919133305
Epoch 1/1
730/730 [==============================] - 12s 16ms/step
0.49782890881162956 0.683561650245157
Epoch 1/1
730/730 [==============================] - 12s 16ms/step
0.505446560782929 0.789041105402659
302/302 [==============================] - 2s 6ms/step
0.386889719694162 0.7086092822303046


In [59]:
y_pred = model.predict(data_sub_4, verbose=0)
y_pred = np.array([int(round(i[0])) for i in y_pred])

from sklearn.metrics import confusion_matrix
confusion_matrix(labels_sub_4,y_pred)

array([[214,  88],
       [  0,   0]])

In [61]:
data_training = np.concatenate((data_sub_2, data_sub_1, data_sub_3, data_sub_5,
                                data_sub_6, data_sub_7, data_sub_8 , data_sub_9, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_2, labels_sub_1, labels_sub_3, labels_sub_5, 
                                  labels_sub_6, labels_sub_7, labels_sub_8, labels_sub_9, labels_sub_10))


def load_model():
    
    IMG_SIZE = 100
    model = Sequential()
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(100, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation = "sigmoid"))

    model.compile(
     optimizer = "adam",
     loss = "binary_crossentropy",
     metrics = ["accuracy"]
    )
    return model

def train_evaluate(model, data_train, labels_train, data_test, labels_test):
    model.fit(x=data_train, y=labels_train, batch_size=5, epochs=1,
         verbose=1, shuffle=True)
    loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)
    print(loss, acc)

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix

if __name__ == "__main__":
    load_model()
    kFold = StratifiedKFold(n_splits=5, shuffle = True)
    for train, test in kFold.split(data_training,labels_training):
        model = None
        model = load_model()
        train_evaluate(model, data_training[train], labels_training[train], data_training[test], labels_training[test])

        
    loss, acc = model.evaluate(x=data_sub_4, y=labels_sub_4, batch_size=5, verbose=1)
    print(loss, acc)
    y_pred = model.predict(data_sub_4, verbose=0)
    y_pred = np.array([int(round(i[0])) for i in y_pred])
    confusion_matrix(labels_sub_4,y_pred)

Epoch 1/1
731/731 [==============================] - 12s 17ms/step
0.5844429868149855 0.6867305145544165
Epoch 1/1
731/731 [==============================] - 12s 17ms/step
0.6712380012066681 0.5458276342554719
Epoch 1/1
731/731 [==============================] - 13s 17ms/step
0.7445494609231336 0.5294117659493445
Epoch 1/1
730/730 [==============================] - 13s 17ms/step
0.6209113312094179 0.6000000046540613
Epoch 1/1
730/730 [==============================] - 13s 17ms/step
0.6001752363900615 0.7178082283433169
302/302 [==============================] - 2s 7ms/step
0.5102799287102869 0.6324503451388404


## Subject 5

In [ ]:
data_training = np.concatenate((data_sub_2, data_sub_1, data_sub_4, data_sub_3,
                                data_sub_6, data_sub_7, data_sub_8 , data_sub_9, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_2, labels_sub_1, labels_sub_4, labels_sub_3, 
                                  labels_sub_6, labels_sub_7, labels_sub_8, labels_sub_9, labels_sub_10))

limit = math.ceil(len(data_training)/len(data_sub_5))

for i in range(limit):
    data_test = np.concatenate((data_sub_5, data_sub_5)) 
    labels_test = np.concatenate((labels_sub_5, labels_sub_5))
    
IMG_SIZE = 100
model = Sequential()



model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(100, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(50, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(1, activation = "softmax"))

model.compile(
 optimizer = "adam",
 loss = "binary_crossentropy",
 metrics = ["accuracy"]
)

model.fit(x=data_training, y=labels_training, batch_size=5, epochs=1,
     verbose=1, shuffle=True)
loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)

print(loss, acc)

In [62]:
data_training = np.concatenate((data_sub_2, data_sub_1, data_sub_3, data_sub_4,
                                data_sub_6, data_sub_7, data_sub_8 , data_sub_9, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_2, labels_sub_1, labels_sub_3, labels_sub_4, 
                                  labels_sub_6, labels_sub_7, labels_sub_8, labels_sub_9, labels_sub_10))


def load_model():
    
    IMG_SIZE = 100
    model = Sequential()
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(100, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation = "sigmoid"))

    model.compile(
     optimizer = "adam",
     loss = "binary_crossentropy",
     metrics = ["accuracy"]
    )
    return model

def train_evaluate(model, data_train, labels_train, data_test, labels_test):
    model.fit(x=data_train, y=labels_train, batch_size=5, epochs=1,
         verbose=1, shuffle=True)
    loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)
    print(loss, acc)

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix

if __name__ == "__main__":
    load_model()
    kFold = StratifiedKFold(n_splits=5, shuffle = True)
    for train, test in kFold.split(data_training,labels_training):
        model = None
        model = load_model()
        train_evaluate(model, data_training[train], labels_training[train], data_training[test], labels_training[test])

        
    loss, acc = model.evaluate(x=data_sub_5, y=labels_sub_5, batch_size=5, verbose=1)
    print(loss, acc)

Epoch 1/1
696/696 [==============================] - 12s 17ms/step
0.1545460498562262 0.9554597722345043
Epoch 1/1
696/696 [==============================] - 12s 17ms/step
0.22372629161222957 0.9022988558843218
Epoch 1/1
695/695 [==============================] - 12s 18ms/step
0.2152846124184003 0.919424464162305
Epoch 1/1
695/695 [==============================] - 12s 18ms/step
0.20593292792763213 0.9194244652343311
Epoch 1/1
695/695 [==============================] - 12s 18ms/step
0.16663728750699447 0.9309352553791279
478/478 [==============================] - 3s 6ms/step
1.0490362716489958 0.5899581623937794


In [64]:
y_pred = model.predict(data_sub_5, verbose=0)
y_pred = np.array([int(round(i[0])) for i in y_pred])
confusion_matrix(labels_sub_5,y_pred)

array([[225,  10],
       [186,  57]])

In [63]:
data_training = np.concatenate((data_sub_2, data_sub_1, data_sub_4, data_sub_3,
                                data_sub_6, data_sub_7, data_sub_8 , data_sub_9, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_2, labels_sub_1, labels_sub_4, labels_sub_3, 
                                  labels_sub_6, labels_sub_7, labels_sub_8, labels_sub_9, labels_sub_10))

limit = math.ceil(len(data_training)/len(data_sub_5))

for i in range(limit):
    data_test = np.concatenate((data_sub_5, data_sub_5)) 
    labels_test = np.concatenate((labels_sub_5, labels_sub_5))
    
IMG_SIZE = 100
model = Sequential()



model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(100, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(50, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(1, activation = "softmax"))

model.compile(
 optimizer = "adam",
 loss = "binary_crossentropy",
 metrics = ["accuracy"]
)

model.fit(x=data_training, y=labels_training, batch_size=5, epochs=1,
     verbose=1, shuffle=True)
loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)

print(loss, acc)

NameError: name 'math' is not defined

In [65]:
data_training = np.concatenate((data_sub_2, data_sub_1, data_sub_3, data_sub_4,
                                data_sub_6, data_sub_7, data_sub_8 , data_sub_9, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_2, labels_sub_1, labels_sub_3, labels_sub_4, 
                                  labels_sub_6, labels_sub_7, labels_sub_8, labels_sub_9, labels_sub_10))


def load_model():
    
    IMG_SIZE = 100
    model = Sequential()
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(100, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation = "sigmoid"))

    model.compile(
     optimizer = "adam",
     loss = "binary_crossentropy",
     metrics = ["accuracy"]
    )
    return model

def train_evaluate(model, data_train, labels_train, data_test, labels_test):
    model.fit(x=data_train, y=labels_train, batch_size=5, epochs=1,
         verbose=1, shuffle=True)
    loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)
    print(loss, acc)

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix

if __name__ == "__main__":
    load_model()
    kFold = StratifiedKFold(n_splits=5, shuffle = True)
    for train, test in kFold.split(data_training,labels_training):
        model = None
        model = load_model()
        train_evaluate(model, data_training[train], labels_training[train], data_training[test], labels_training[test])

        
    loss, acc = model.evaluate(x=data_sub_5, y=labels_sub_5, batch_size=5, verbose=1)
    print(loss, acc)

Epoch 1/1
696/696 [==============================] - 14s 20ms/step
0.5313664184262356 0.7557471361996113
Epoch 1/1
696/696 [==============================] - 14s 20ms/step
0.39928225697628383 0.850574718873905
Epoch 1/1
695/695 [==============================] - 14s 20ms/step
0.4755306840813953 0.6920863348159859
Epoch 1/1
695/695 [==============================] - 14s 20ms/step
0.49833015411448994 0.7597122370339126
Epoch 1/1
695/695 [==============================] - 14s 20ms/step
0.431423962303846 0.8057554040452559
478/478 [==============================] - 3s 7ms/step
0.43955164000579006 0.8368200909143712


In [66]:
y_pred = model.predict(data_sub_5, verbose=0)
y_pred = np.array([int(round(i[0])) for i in y_pred])
confusion_matrix(labels_sub_5,y_pred)

array([[172,  63],
       [ 15, 228]])

In [69]:
data_training = np.concatenate((data_sub_2, data_sub_1, data_sub_3, data_sub_4,
                                data_sub_6, data_sub_7, data_sub_8 , data_sub_9, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_2, labels_sub_1, labels_sub_3, labels_sub_4, 
                                  labels_sub_6, labels_sub_7, labels_sub_8, labels_sub_9, labels_sub_10))


def load_model():
    
    IMG_SIZE = 100
    model = Sequential()
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(100, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation = "sigmoid"))

    model.compile(
     optimizer = "adam",
     loss = "binary_crossentropy",
     metrics = ["accuracy"]
    )
    return model

def train_evaluate(model, data_train, labels_train, data_test, labels_test):
    model.fit(x=data_train, y=labels_train, batch_size=5, epochs=1,
         verbose=1, shuffle=True)
    loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)
    print(loss, acc)

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix

if __name__ == "__main__":
    load_model()
    kFold = StratifiedKFold(n_splits=5, shuffle = True)
    for train, test in kFold.split(data_training,labels_training):
        model = None
        model = load_model()
        train_evaluate(model, data_training[train], labels_training[train], data_training[test], labels_training[test])

        
    loss, acc = model.evaluate(x=data_sub_5, y=labels_sub_5, batch_size=5, verbose=1)
    print(loss, acc)

Epoch 1/1
696/696 [==============================] - 15s 21ms/step
0.5567711929405301 0.6307471296054193
Epoch 1/1
696/696 [==============================] - 15s 21ms/step
0.5577037953017293 0.642241390464806
Epoch 1/1
695/695 [==============================] - 15s 21ms/step
0.6042864191875184 0.5755395692029446
Epoch 1/1
695/695 [==============================] - 15s 22ms/step
0.6277015134156179 0.5755395690957419
Epoch 1/1
695/695 [==============================] - 15s 22ms/step
0.6247443636115506 0.5755395692029446
478/478 [==============================] - 3s 7ms/step
0.601888292504155 0.49163179922552785


In [70]:
y_pred = model.predict(data_sub_5, verbose=0)
y_pred = np.array([int(round(i[0])) for i in y_pred])
confusion_matrix(labels_sub_5,y_pred)

array([[235,   0],
       [243,   0]])

## Subject 6

In [72]:
data_training = np.concatenate((data_sub_2, data_sub_1, data_sub_4, data_sub_5,
                                data_sub_3, data_sub_7, data_sub_8 , data_sub_9, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_2, labels_sub_1, labels_sub_4, labels_sub_5, 
                                  labels_sub_3, labels_sub_7, labels_sub_8, labels_sub_9, labels_sub_10))


data_test = np.concatenate((data_sub_6, data_sub_6)) 
labels_test = np.concatenate((labels_sub_6, labels_sub_6))
    
IMG_SIZE = 100
model = Sequential()

model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(100, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(50, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(1, activation = "softmax"))

model.compile(
 optimizer = "adam",
 loss = "binary_crossentropy",
 metrics = ["accuracy"]
)

model.fit(x=data_training, y=labels_training, batch_size=5, epochs=1,
     verbose=1, shuffle=True)
loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)

print(loss, acc)

Epoch 1/1
990/990 [==============================] - 16s 16ms/step
8.116123189829827 0.49090909114991776


In [73]:
data_training = np.concatenate((data_sub_2, data_sub_1, data_sub_3, data_sub_4,
                                data_sub_5, data_sub_7, data_sub_8 , data_sub_9, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_2, labels_sub_1, labels_sub_3, labels_sub_4, 
                                  labels_sub_5, labels_sub_7, labels_sub_8, labels_sub_9, labels_sub_10))


def load_model():
    
    IMG_SIZE = 100
    model = Sequential()
    
    
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(100, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation = "sigmoid"))

    model.compile(
     optimizer = "adam",
     loss = "binary_crossentropy",
     metrics = ["accuracy"]
    )
    return model

def train_evaluate(model, data_train, labels_train, data_test, labels_test):
    model.fit(x=data_train, y=labels_train, batch_size=5, epochs=1,
         verbose=1, shuffle=True)
    loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)
    print(loss, acc)

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix


load_model()
kFold = StratifiedKFold(n_splits=5, shuffle = True)
for train, test in kFold.split(data_training,labels_training):
    model = None
    model = load_model()
    train_evaluate(model, data_training[train], labels_training[train], data_training[test], labels_training[test])


loss, acc = model.evaluate(x=data_sub_6, y=labels_sub_6, batch_size=5, verbose=1)
print(loss, acc)
y_pred = model.predict(data_sub_6, verbose=0)
y_pred = np.array([int(round(i[0])) for i in y_pred])
confusion_matrix(labels_sub_6,y_pred)

Epoch 1/1
692/692 [==============================] - 15s 21ms/step
0.31704180719293223 0.8641618564078918
Epoch 1/1
692/692 [==============================] - 15s 21ms/step
0.24930735081150782 0.9017341078792004
Epoch 1/1
692/692 [==============================] - 15s 21ms/step
0.23345203719214896 0.9089595420296827
Epoch 1/1
692/692 [==============================] - 15s 22ms/step
0.3216666115156504 0.8627167690351519
Epoch 1/1
692/692 [==============================] - 15s 22ms/step
0.24690680307382584 0.9031791956610762
495/495 [==============================] - 3s 6ms/step
2.068297872785479 0.5090909118604179


array([[234,  18],
       [225,  18]])

In [ ]:
data_training = np.concatenate((data_sub_2, data_sub_1, data_sub_4, data_sub_5,
                                data_sub_6, data_sub_7, data_sub_8 , data_sub_9, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_2, labels_sub_1, labels_sub_4, labels_sub_5, 
                                  labels_sub_6, labels_sub_7, labels_sub_8, labels_sub_9, labels_sub_10))

limit = math.ceil(len(data_training)/len(data_sub_3))

for i in range(limit):
    data_test = np.concatenate((data_sub_3, data_sub_3)) 
    labels_test = np.concatenate((labels_sub_3, labels_sub_3))
    
IMG_SIZE = 100
model = Sequential()



model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(100, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(50, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(1, activation = "softmax"))

model.compile(
 optimizer = "adam",
 loss = "binary_crossentropy",
 metrics = ["accuracy"]
)

model.fit(x=data_training, y=labels_training, batch_size=5, epochs=1,
     verbose=1, shuffle=True)
loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)

print(loss, acc)

In [74]:
data_training = np.concatenate((data_sub_2, data_sub_1, data_sub_3, data_sub_4,
                                data_sub_5, data_sub_7, data_sub_8 , data_sub_9, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_2, labels_sub_1, labels_sub_3, labels_sub_4, 
                                  labels_sub_5, labels_sub_7, labels_sub_8, labels_sub_9, labels_sub_10))


def load_model():
    
    IMG_SIZE = 100
    model = Sequential()
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(100, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation = "sigmoid"))

    model.compile(
     optimizer = "adam",
     loss = "binary_crossentropy",
     metrics = ["accuracy"]
    )
    return model

def train_evaluate(model, data_train, labels_train, data_test, labels_test):
    model.fit(x=data_train, y=labels_train, batch_size=5, epochs=1,
         verbose=1, shuffle=True)
    loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)
    print(loss, acc)

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix


load_model()
kFold = StratifiedKFold(n_splits=5, shuffle = True)
for train, test in kFold.split(data_training,labels_training):
    model = None
    model = load_model()
    train_evaluate(model, data_training[train], labels_training[train], data_training[test], labels_training[test])


loss, acc = model.evaluate(x=data_sub_6, y=labels_sub_6, batch_size=5, verbose=1)
print(loss, acc)
y_pred = model.predict(data_sub_6, verbose=0)
y_pred = np.array([int(round(i[0])) for i in y_pred])
confusion_matrix(labels_sub_6,y_pred)

Epoch 1/1
692/692 [==============================] - 16s 23ms/step
0.4936970218765749 0.7413294889635778
Epoch 1/1
692/692 [==============================] - 16s 23ms/step
0.508745660152787 0.7095375766686965
Epoch 1/1
692/692 [==============================] - 16s 24ms/step
0.5038750962531894 0.6502890199896573
Epoch 1/1
692/692 [==============================] - 16s 24ms/step
0.5133207749112407 0.7369942274913622
Epoch 1/1
692/692 [==============================] - 19s 27ms/step
0.4430420984281227 0.8338150364815156
495/495 [==============================] - 4s 7ms/step
0.9747477201274549 0.4848484913207064


array([[204,  48],
       [207,  36]])

In [75]:
data_training = np.concatenate((data_sub_2, data_sub_1, data_sub_3, data_sub_4,
                                data_sub_5, data_sub_7, data_sub_8 , data_sub_9, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_2, labels_sub_1, labels_sub_3, labels_sub_4, 
                                  labels_sub_5, labels_sub_7, labels_sub_8, labels_sub_9, labels_sub_10))


def load_model():
    
    IMG_SIZE = 100
    model = Sequential()
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(100, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation = "sigmoid"))

    model.compile(
     optimizer = "adam",
     loss = "binary_crossentropy",
     metrics = ["accuracy"]
    )
    return model

def train_evaluate(model, data_train, labels_train, data_test, labels_test):
    model.fit(x=data_train, y=labels_train, batch_size=5, epochs=1,
         verbose=1, shuffle=True)
    loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)
    print(loss, acc)

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix


load_model()
kFold = StratifiedKFold(n_splits=5, shuffle = True)
for train, test in kFold.split(data_training,labels_training):
    model = None
    model = load_model()
    train_evaluate(model, data_training[train], labels_training[train], data_training[test], labels_training[test])


loss, acc = model.evaluate(x=data_sub_6, y=labels_sub_6, batch_size=5, verbose=1)
print(loss, acc)
y_pred = model.predict(data_sub_6, verbose=0)
y_pred = np.array([int(round(i[0])) for i in y_pred])
confusion_matrix(labels_sub_6,y_pred)

Epoch 1/1
692/692 [==============================] - 17s 25ms/step
0.5903730901291949 0.573699422611322
Epoch 1/1
692/692 [==============================] - 18s 25ms/step
0.6515888631171574 0.573699422611322
Epoch 1/1
692/692 [==============================] - 17s 25ms/step
0.5264895996792145 0.6026011571460377
Epoch 1/1
692/692 [==============================] - 18s 25ms/step
0.5861526251140702 0.5939306373239597
Epoch 1/1
692/692 [==============================] - 22s 32ms/step
0.4817157610712541 0.7890173485987104
495/495 [==============================] - 4s 8ms/step
0.9321833527599922 0.5151515235804548


array([[192,  60],
       [180,  63]])

In [76]:
data_training = np.concatenate((data_sub_2, data_sub_1, data_sub_3, data_sub_4,
                                data_sub_5, data_sub_7, data_sub_8 , data_sub_9, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_2, labels_sub_1, labels_sub_3, labels_sub_4, 
                                  labels_sub_5, labels_sub_7, labels_sub_8, labels_sub_9, labels_sub_10))


def load_model():
    
    IMG_SIZE = 100
    model = Sequential()
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(100, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation = "sigmoid"))

    model.compile(
     optimizer = "adam",
     loss = "binary_crossentropy",
     metrics = ["accuracy"]
    )
    return model

def train_evaluate(model, data_train, labels_train, data_test, labels_test):
    model.fit(x=data_train, y=labels_train, batch_size=5, epochs=1,
         verbose=1, shuffle=True)
    loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)
    print(loss, acc)

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix


load_model()
kFold = StratifiedKFold(n_splits=5, shuffle = True)
for train, test in kFold.split(data_training,labels_training):
    model = None
    model = load_model()
    train_evaluate(model, data_training[train], labels_training[train], data_training[test], labels_training[test])


loss, acc = model.evaluate(x=data_sub_6, y=labels_sub_6, batch_size=5, verbose=1)
print(loss, acc)
y_pred = model.predict(data_sub_6, verbose=0)
y_pred = np.array([int(round(i[0])) for i in y_pred])
confusion_matrix(labels_sub_6,y_pred)

Epoch 1/1
  20/2768 [..............................] - ETA: 1:28:33 - loss: 0.9089 - acc: 0.5500

/home/aniketh/.local/lib/python3.6/site-packages/keras/callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.383740). Check your callbacks.
  % delta_t_median)


692/692 [==============================] - 20s 29ms/step
0.6751169718242105 0.5736994228266568
Epoch 1/1
692/692 [==============================] - 18s 26ms/step
0.5917933481621604 0.6647398929849181
Epoch 1/1
692/692 [==============================] - 18s 27ms/step
0.6177328314223042 0.6791907598216065
Epoch 1/1
692/692 [==============================] - 19s 27ms/step
0.5491762025809357 0.6921965391778877
Epoch 1/1
692/692 [==============================] - 19s 27ms/step
0.6865103575535593 0.5289017372479329
495/495 [==============================] - 4s 8ms/step
0.6945596054346874 0.4646464747310889


array([[ 68, 184],
       [ 81, 162]])

## Subject 7

In [ ]:
data_training = np.concatenate((data_sub_2, data_sub_1, data_sub_4, data_sub_5,
                                data_sub_6, data_sub_7, data_sub_8 , data_sub_9, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_2, labels_sub_1, labels_sub_4, labels_sub_5, 
                                  labels_sub_6, labels_sub_7, labels_sub_8, labels_sub_9, labels_sub_10))

limit = math.ceil(len(data_training)/len(data_sub_3))

for i in range(limit):
    data_test = np.concatenate((data_sub_3, data_sub_3)) 
    labels_test = np.concatenate((labels_sub_3, labels_sub_3))
    
IMG_SIZE = 100
model = Sequential()



model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(100, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(50, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(1, activation = "softmax"))

model.compile(
 optimizer = "adam",
 loss = "binary_crossentropy",
 metrics = ["accuracy"]
)

model.fit(x=data_training, y=labels_training, batch_size=5, epochs=3,
     verbose=1, shuffle=True)
loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)

print(loss, acc)

In [77]:
data_training = np.concatenate((data_sub_2, data_sub_1, data_sub_3, data_sub_4,
                                data_sub_5, data_sub_6, data_sub_8 , data_sub_9, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_2, labels_sub_1, labels_sub_3, labels_sub_4, 
                                  labels_sub_5, labels_sub_6, labels_sub_8, labels_sub_9, labels_sub_10))


def load_model():
    
    IMG_SIZE = 100
    model = Sequential()
    
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(100, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation = "sigmoid"))

    model.compile(
     optimizer = "adam",
     loss = "binary_crossentropy",
     metrics = ["accuracy"]
    )
    return model

def train_evaluate(model, data_train, labels_train, data_test, labels_test):
    model.fit(x=data_train, y=labels_train, batch_size=5, epochs=1,
         verbose=1, shuffle=True)
    loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)
    print(loss, acc)

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix


load_model()
kFold = StratifiedKFold(n_splits=5, shuffle = True)
for train, test in kFold.split(data_training,labels_training):
    model = None
    model = load_model()
    train_evaluate(model, data_training[train], labels_training[train], data_training[test], labels_training[test])


loss, acc = model.evaluate(x=data_sub_7, y=labels_sub_7, batch_size=5, verbose=1)
print(loss, acc)
y_pred = model.predict(data_sub_7, verbose=0)
y_pred = np.array([int(round(i[0])) for i in y_pred])
confusion_matrix(labels_sub_7,y_pred)

Epoch 1/1
700/700 [==============================] - 19s 27ms/step
0.30895378320544425 0.8628571466675826
Epoch 1/1
700/700 [==============================] - 19s 27ms/step
0.24595536126954748 0.9042857197778565
Epoch 1/1
699/699 [==============================] - 19s 27ms/step
0.1937829755075675 0.9241774002881521
Epoch 1/1
699/699 [==============================] - 19s 27ms/step
0.21595856958156842 0.9256080154526728
Epoch 1/1
698/698 [==============================] - 19s 27ms/step
0.22398459237455162 0.9068767957410703
459/459 [==============================] - 3s 6ms/step
0.684788477576636 0.5773420572800314


array([[174,  51],
       [143,  91]])

In [ ]:
data_training = np.concatenate((data_sub_2, data_sub_1, data_sub_4, data_sub_5,
                                data_sub_6, data_sub_7, data_sub_8 , data_sub_9, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_2, labels_sub_1, labels_sub_4, labels_sub_5, 
                                  labels_sub_6, labels_sub_7, labels_sub_8, labels_sub_9, labels_sub_10))

limit = math.ceil(len(data_training)/len(data_sub_3))

for i in range(limit):
    data_test = np.concatenate((data_sub_3, data_sub_3)) 
    labels_test = np.concatenate((labels_sub_3, labels_sub_3))
    
IMG_SIZE = 100
model = Sequential()



model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(100, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(50, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(1, activation = "softmax"))

model.compile(
 optimizer = "adam",
 loss = "binary_crossentropy",
 metrics = ["accuracy"]
)

model.fit(x=data_training, y=labels_training, batch_size=5, epochs=3,
     verbose=1, shuffle=True)
loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)

print(loss, acc)

In [78]:
data_training = np.concatenate((data_sub_2, data_sub_1, data_sub_3, data_sub_4,
                                data_sub_5, data_sub_6, data_sub_8 , data_sub_9, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_2, labels_sub_1, labels_sub_3, labels_sub_4, 
                                  labels_sub_5, labels_sub_6, labels_sub_8, labels_sub_9, labels_sub_10))


def load_model():
    
    IMG_SIZE = 100
    model = Sequential()
    
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(100, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation = "sigmoid"))

    model.compile(
     optimizer = "adam",
     loss = "binary_crossentropy",
     metrics = ["accuracy"]
    )
    return model

def train_evaluate(model, data_train, labels_train, data_test, labels_test):
    model.fit(x=data_train, y=labels_train, batch_size=5, epochs=1,
         verbose=1, shuffle=True)
    loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)
    print(loss, acc)

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix


load_model()
kFold = StratifiedKFold(n_splits=5, shuffle = True)
for train, test in kFold.split(data_training,labels_training):
    model = None
    model = load_model()
    train_evaluate(model, data_training[train], labels_training[train], data_training[test], labels_training[test])


loss, acc = model.evaluate(x=data_sub_7, y=labels_sub_7, batch_size=5, verbose=1)
print(loss, acc)
y_pred = model.predict(data_sub_7, verbose=0)
y_pred = np.array([int(round(i[0])) for i in y_pred])
confusion_matrix(labels_sub_7,y_pred)

Epoch 1/1
700/700 [==============================] - 21s 30ms/step
0.5461762206895011 0.6971428645508629
Epoch 1/1
700/700 [==============================] - 20s 29ms/step
0.49628514514437744 0.7557142909084048
Epoch 1/1
699/699 [==============================] - 20s 29ms/step
0.46343233577043713 0.748211739933883
Epoch 1/1
699/699 [==============================] - 20s 29ms/step
0.4674393598027581 0.8040057322455407
Epoch 1/1
698/698 [==============================] - 21s 30ms/step
0.5128757234203781 0.6934097486529446
459/459 [==============================] - 3s 7ms/step
0.6181746874499685 0.49891068109499864


array([[190,  35],
       [195,  39]])

In [3]:
data_training = np.concatenate((data_sub_2, data_sub_1, data_sub_3, data_sub_4,
                                data_sub_5, data_sub_6, data_sub_8 , data_sub_9, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_2, labels_sub_1, labels_sub_3, labels_sub_4, 
                                  labels_sub_5, labels_sub_6, labels_sub_8, labels_sub_9, labels_sub_10))


def load_model():
    
    IMG_SIZE = 100
    model = Sequential()
    
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(100, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation = "sigmoid"))

    model.compile(
     optimizer = "adam",
     loss = "binary_crossentropy",
     metrics = ["accuracy"]
    )
    return model

def train_evaluate(model, data_train, labels_train, data_test, labels_test):
    model.fit(x=data_train, y=labels_train, batch_size=5, epochs=1,
         verbose=1, shuffle=True)
    loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)
    print(loss, acc)

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix


load_model()
kFold = StratifiedKFold(n_splits=5, shuffle = True)
for train, test in kFold.split(data_training,labels_training):
    model = None
    model = load_model()
    train_evaluate(model, data_training[train], labels_training[train], data_training[test], labels_training[test])


loss, acc = model.evaluate(x=data_sub_7, y=labels_sub_7, batch_size=5, verbose=1)
print(loss, acc)
y_pred = model.predict(data_sub_7, verbose=0)
y_pred = np.array([int(round(i[0])) for i in y_pred])
confusion_matrix(labels_sub_7,y_pred)

Epoch 1/1
700/700 [==============================] - 4s 6ms/step
0.5558356375566551 0.5757142862038953
Epoch 1/1
700/700 [==============================] - 4s 6ms/step
0.5146180419517415 0.7471428667860371
Epoch 1/1
699/699 [==============================] - 4s 6ms/step
0.5678224484296316 0.6294706785499794
Epoch 1/1
699/699 [==============================] - 5s 6ms/step
0.5332219218022493 0.7281831275026857
Epoch 1/1
698/698 [==============================] - 5s 7ms/step
0.5970975378031887 0.6146131854472324
459/459 [==============================] - 3s 6ms/step
0.5162168633119733 0.7320261542443876


array([[193,  32],
       [ 91, 143]])

In [4]:
data_training = np.concatenate((data_sub_2, data_sub_1, data_sub_3, data_sub_4,
                                data_sub_5, data_sub_6, data_sub_8 , data_sub_9, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_2, labels_sub_1, labels_sub_3, labels_sub_4, 
                                  labels_sub_5, labels_sub_6, labels_sub_8, labels_sub_9, labels_sub_10))


def load_model():
    
    IMG_SIZE = 100
    model = Sequential()
    
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(100, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation = "sigmoid"))

    model.compile(
     optimizer = "adam",
     loss = "binary_crossentropy",
     metrics = ["accuracy"]
    )
    return model

def train_evaluate(model, data_train, labels_train, data_test, labels_test):
    model.fit(x=data_train, y=labels_train, batch_size=5, epochs=1,
         verbose=1, shuffle=True)
    loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)
    print(loss, acc)

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix


load_model()
kFold = StratifiedKFold(n_splits=5, shuffle = True)
for train, test in kFold.split(data_training,labels_training):
    model = None
    model = load_model()
    train_evaluate(model, data_training[train], labels_training[train], data_training[test], labels_training[test])


loss, acc = model.evaluate(x=data_sub_7, y=labels_sub_7, batch_size=5, verbose=1)
print(loss, acc)
y_pred = model.predict(data_sub_7, verbose=0)
y_pred = np.array([int(round(i[0])) for i in y_pred])
confusion_matrix(labels_sub_7,y_pred)

Epoch 1/1
700/700 [==============================] - 5s 7ms/step
0.6808065452745983 0.5757142863103322
Epoch 1/1
700/700 [==============================] - 5s 7ms/step
0.6838896429964474 0.5757142868425165
Epoch 1/1
699/699 [==============================] - 5s 7ms/step
0.7880655812560574 0.5751072967342382
Epoch 1/1
699/699 [==============================] - 5s 7ms/step
0.6858353231512596 0.5751072967342382
Epoch 1/1
698/698 [==============================] - 5s 7ms/step
0.6812407276179524 0.575931232924277
459/459 [==============================] - 3s 7ms/step
0.7054857657106354 0.49019607846383695


array([[225,   0],
       [234,   0]])

## Subject 8

In [ ]:
data_training = np.concatenate((data_sub_2, data_sub_1, data_sub_4, data_sub_5,
                                data_sub_6, data_sub_7, data_sub_8 , data_sub_9, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_2, labels_sub_1, labels_sub_4, labels_sub_5, 
                                  labels_sub_6, labels_sub_7, labels_sub_8, labels_sub_9, labels_sub_10))

limit = math.ceil(len(data_training)/len(data_sub_3))

for i in range(limit):
    data_test = np.concatenate((data_sub_3, data_sub_3)) 
    labels_test = np.concatenate((labels_sub_3, labels_sub_3))
    
IMG_SIZE = 100
model = Sequential()



model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(100, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(50, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(1, activation = "softmax"))

model.compile(
 optimizer = "adam",
 loss = "binary_crossentropy",
 metrics = ["accuracy"]
)

model.fit(x=data_training, y=labels_training, batch_size=5, epochs=3,
     verbose=1, shuffle=True)
loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)

print(loss, acc)

In [5]:
data_training = np.concatenate((data_sub_2, data_sub_1, data_sub_3, data_sub_4,
                                data_sub_5, data_sub_6, data_sub_7 , data_sub_9, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_2, labels_sub_1, labels_sub_3, labels_sub_4, 
                                  labels_sub_5, labels_sub_6, labels_sub_7, labels_sub_9, labels_sub_10))


def load_model():
    
    IMG_SIZE = 100
    model = Sequential()
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(100, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation = "sigmoid"))

    model.compile(
     optimizer = "adam",
     loss = "binary_crossentropy",
     metrics = ["accuracy"]
    )
    return model

def train_evaluate(model, data_train, labels_train, data_test, labels_test):
    model.fit(x=data_train, y=labels_train, batch_size=5, epochs=1,
         verbose=1, shuffle=True)
    loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)
    print(loss, acc)

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix


load_model()
kFold = StratifiedKFold(n_splits=5, shuffle = True)
for train, test in kFold.split(data_training,labels_training):
    model = None
    model = load_model()
    train_evaluate(model, data_training[train], labels_training[train], data_training[test], labels_training[test])


loss, acc = model.evaluate(x=data_sub_8, y=labels_sub_8, batch_size=5, verbose=1)
print(loss, acc)
y_pred = model.predict(data_sub_8, verbose=0)
y_pred = np.array([int(round(i[0])) for i in y_pred])
confusion_matrix(labels_sub_8,y_pred)

Epoch 1/1
752/752 [==============================] - 4s 6ms/step
0.16368959114682163 0.9335106421816857
Epoch 1/1
752/752 [==============================] - 5s 6ms/step
0.16997288191603419 0.9414893649914797
Epoch 1/1
751/751 [==============================] - 5s 6ms/step
0.16461088899436213 0.9374167812807106
Epoch 1/1
751/751 [==============================] - 5s 6ms/step
0.2618427143522728 0.8801597936969305
Epoch 1/1
751/751 [==============================] - 5s 6ms/step
0.18343567771489264 0.936085223516675
198/198 [==============================] - 1s 6ms/step
1.2545231571326954 0.45959596433723815


array([[79, 11],
       [96, 12]])

In [ ]:
data_training = np.concatenate((data_sub_2, data_sub_1, data_sub_4, data_sub_5,
                                data_sub_6, data_sub_7, data_sub_8 , data_sub_9, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_2, labels_sub_1, labels_sub_4, labels_sub_5, 
                                  labels_sub_6, labels_sub_7, labels_sub_8, labels_sub_9, labels_sub_10))

limit = math.ceil(len(data_training)/len(data_sub_3))

for i in range(limit):
    data_test = np.concatenate((data_sub_3, data_sub_3)) 
    labels_test = np.concatenate((labels_sub_3, labels_sub_3))
    
IMG_SIZE = 100
model = Sequential()



model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(100, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(50, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(1, activation = "softmax"))

model.compile(
 optimizer = "adam",
 loss = "binary_crossentropy",
 metrics = ["accuracy"]
)

model.fit(x=data_training, y=labels_training, batch_size=5, epochs=3,
     verbose=1, shuffle=True)
loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)

print(loss, acc)

In [6]:
data_training = np.concatenate((data_sub_2, data_sub_1, data_sub_3, data_sub_4,
                                data_sub_5, data_sub_6, data_sub_7 , data_sub_9, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_2, labels_sub_1, labels_sub_3, labels_sub_4, 
                                  labels_sub_5, labels_sub_6, labels_sub_7, labels_sub_9, labels_sub_10))


def load_model():
    
    IMG_SIZE = 100
    model = Sequential()
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(100, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation = "sigmoid"))

    model.compile(
     optimizer = "adam",
     loss = "binary_crossentropy",
     metrics = ["accuracy"]
    )
    return model

def train_evaluate(model, data_train, labels_train, data_test, labels_test):
    model.fit(x=data_train, y=labels_train, batch_size=5, epochs=1,
         verbose=1, shuffle=True)
    loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)
    print(loss, acc)

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix


load_model()
kFold = StratifiedKFold(n_splits=5, shuffle = True)
for train, test in kFold.split(data_training,labels_training):
    model = None
    model = load_model()
    train_evaluate(model, data_training[train], labels_training[train], data_training[test], labels_training[test])


loss, acc = model.evaluate(x=data_sub_8, y=labels_sub_8, batch_size=5, verbose=1)
print(loss, acc)
y_pred = model.predict(data_sub_8, verbose=0)
y_pred = np.array([int(round(i[0])) for i in y_pred])
confusion_matrix(labels_sub_8,y_pred)

Epoch 1/1
752/752 [==============================] - 6s 7ms/step
0.47024956651334154 0.7726063881109052
Epoch 1/1
752/752 [==============================] - 6s 7ms/step
0.41966782746517023 0.7938829885716451
Epoch 1/1
751/751 [==============================] - 6s 8ms/step
0.51851703606536 0.7190412887918965
Epoch 1/1
751/751 [==============================] - 6s 8ms/step
0.4979268148263666 0.7589880236971395
Epoch 1/1
751/751 [==============================] - 6s 8ms/step
0.4361662690613925 0.7523302302141481
198/198 [==============================] - 1s 7ms/step
0.7817663819049344 0.44949494994649986


array([[ 89,   1],
       [108,   0]])

In [7]:
data_training = np.concatenate((data_sub_2, data_sub_1, data_sub_3, data_sub_4,
                                data_sub_5, data_sub_6, data_sub_7 , data_sub_9, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_2, labels_sub_1, labels_sub_3, labels_sub_4, 
                                  labels_sub_5, labels_sub_6, labels_sub_7, labels_sub_9, labels_sub_10))


def load_model():
    
    IMG_SIZE = 100
    model = Sequential()
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(100, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation = "sigmoid"))

    model.compile(
     optimizer = "adam",
     loss = "binary_crossentropy",
     metrics = ["accuracy"]
    )
    return model

def train_evaluate(model, data_train, labels_train, data_test, labels_test):
    model.fit(x=data_train, y=labels_train, batch_size=5, epochs=1,
         verbose=1, shuffle=True)
    loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)
    print(loss, acc)

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix


load_model()
kFold = StratifiedKFold(n_splits=5, shuffle = True)
for train, test in kFold.split(data_training,labels_training):
    model = None
    model = load_model()
    train_evaluate(model, data_training[train], labels_training[train], data_training[test], labels_training[test])


loss, acc = model.evaluate(x=data_sub_8, y=labels_sub_8, batch_size=5, verbose=1)
print(loss, acc)
y_pred = model.predict(data_sub_8, verbose=0)
y_pred = np.array([int(round(i[0])) for i in y_pred])
confusion_matrix(labels_sub_8,y_pred)

Epoch 1/1
752/752 [==============================] - 6s 8ms/step
0.5824044578054801 0.5718085113912821
Epoch 1/1
752/752 [==============================] - 6s 8ms/step
0.5299788260971137 0.6981383061749821
Epoch 1/1
751/751 [==============================] - 6s 8ms/step
0.5313540811342907 0.580559254922816
Epoch 1/1
751/751 [==============================] - 6s 8ms/step
0.5551252430986945 0.7709720460735212
Epoch 1/1
751/751 [==============================] - 7s 9ms/step
0.5391091385610729 0.6125166467359952
198/198 [==============================] - 1s 7ms/step
0.689685341234159 0.45454545469597135


array([[ 90,   0],
       [108,   0]])

In [11]:
data_training = np.concatenate((data_sub_2, data_sub_1, data_sub_3, data_sub_4,
                                data_sub_5, data_sub_6, data_sub_7 , data_sub_9, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_2, labels_sub_1, labels_sub_3, labels_sub_4, 
                                  labels_sub_5, labels_sub_6, labels_sub_7, labels_sub_9, labels_sub_10))


def load_model():
    
    IMG_SIZE = 100
    model = Sequential()
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(100, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation = "sigmoid"))

    model.compile(
     optimizer = "adam",
     loss = "binary_crossentropy",
     metrics = ["accuracy"]
    )
    return model

def train_evaluate(model, data_train, labels_train, data_test, labels_test):
    model.fit(x=data_train, y=labels_train, batch_size=5, epochs=1,
         verbose=1, shuffle=True)
    loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)
    print(loss, acc)

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix


load_model()
kFold = StratifiedKFold(n_splits=5, shuffle = True)
for train, test in kFold.split(data_training,labels_training):
    model = None
    model = load_model()
    train_evaluate(model, data_training[train], labels_training[train], data_training[test], labels_training[test])


loss, acc = model.evaluate(x=data_sub_8, y=labels_sub_8, batch_size=5, verbose=1)
print(loss, acc)
y_pred = model.predict(data_sub_8, verbose=0)
y_pred = np.array([int(round(i[0])) for i in y_pred])
confusion_matrix(labels_sub_8,y_pred)

Epoch 1/1
752/752 [==============================] - 8s 11ms/step
0.6863450138492787 0.5718085115894358
Epoch 1/1
752/752 [==============================] - 8s 11ms/step
0.6872497473229119 0.5718085115894358
Epoch 1/1
751/751 [==============================] - 8s 11ms/step
0.8856602629991409 0.5712383498205802
Epoch 1/1
751/751 [==============================] - 9s 11ms/step
0.5908562288620818 0.5712383497213714
Epoch 1/1
751/751 [==============================] - 9s 12ms/step
0.674903326202804 0.5712383496221626
198/198 [==============================] - 1s 7ms/step
0.7467299738917688 0.45454545469597135


array([[ 90,   0],
       [108,   0]])

## Subject 9

In [ ]:
data_training = np.concatenate((data_sub_2, data_sub_1, data_sub_4, data_sub_5,
                                data_sub_6, data_sub_7, data_sub_8 , data_sub_9, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_2, labels_sub_1, labels_sub_4, labels_sub_5, 
                                  labels_sub_6, labels_sub_7, labels_sub_8, labels_sub_9, labels_sub_10))

limit = math.ceil(len(data_training)/len(data_sub_3))

for i in range(limit):
    data_test = np.concatenate((data_sub_3, data_sub_3)) 
    labels_test = np.concatenate((labels_sub_3, labels_sub_3))
    
IMG_SIZE = 100
model = Sequential()



model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(100, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(50, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(1, activation = "softmax"))

model.compile(
 optimizer = "adam",
 loss = "binary_crossentropy",
 metrics = ["accuracy"]
)

model.fit(x=data_training, y=labels_training, batch_size=5, epochs=3,
     verbose=1, shuffle=True)
loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)

print(loss, acc)

In [ ]:
data_training = np.concatenate((data_sub_2, data_sub_1, data_sub_4, data_sub_5,
                                data_sub_6, data_sub_7, data_sub_8 , data_sub_9, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_2, labels_sub_1, labels_sub_4, labels_sub_5, 
                                  labels_sub_6, labels_sub_7, labels_sub_8, labels_sub_9, labels_sub_10))

limit = math.ceil(len(data_training)/len(data_sub_3))

for i in range(limit):
    data_test = np.concatenate((data_sub_3, data_sub_3)) 
    labels_test = np.concatenate((labels_sub_3, labels_sub_3))
    
IMG_SIZE = 100
model = Sequential()



model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(100, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(50, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(1, activation = "softmax"))

model.compile(
 optimizer = "adam",
 loss = "binary_crossentropy",
 metrics = ["accuracy"]
)

model.fit(x=data_training, y=labels_training, batch_size=5, epochs=3,
     verbose=1, shuffle=True)
loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)

print(loss, acc)

In [8]:
data_training = np.concatenate((data_sub_2, data_sub_1, data_sub_3, data_sub_4,
                                data_sub_5, data_sub_6, data_sub_7 , data_sub_8, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_2, labels_sub_1, labels_sub_3, labels_sub_4, 
                                  labels_sub_5, labels_sub_6, labels_sub_7, labels_sub_8, labels_sub_10))


def load_model():
    
    IMG_SIZE = 100
    model = Sequential()
    
    
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(100, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation = "sigmoid"))

    model.compile(
     optimizer = "adam",
     loss = "binary_crossentropy",
     metrics = ["accuracy"]
    )
    return model

def train_evaluate(model, data_train, labels_train, data_test, labels_test):
    model.fit(x=data_train, y=labels_train, batch_size=5, epochs=1,
         verbose=1, shuffle=True)
    loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)
    print(loss, acc)

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix


load_model()
kFold = StratifiedKFold(n_splits=5, shuffle = True)
for train, test in kFold.split(data_training,labels_training):
    model = None
    model = load_model()
    train_evaluate(model, data_training[train], labels_training[train], data_training[test], labels_training[test])


loss, acc = model.evaluate(x=data_sub_9, y=labels_sub_9, batch_size=5, verbose=1)
print(loss, acc)
y_pred = model.predict(data_sub_9, verbose=0)
y_pred = np.array([int(round(i[0])) for i in y_pred])
confusion_matrix(labels_sub_9,y_pred)

Epoch 1/1
705/705 [==============================] - 5s 8ms/step
0.16462874548106804 0.9418439739350731
Epoch 1/1
705/705 [==============================] - 5s 8ms/step
0.19416080419377074 0.9205673804097142
Epoch 1/1
705/705 [==============================] - 6s 8ms/step
0.17091983857280607 0.9361702163591452
Epoch 1/1
704/704 [==============================] - 6s 8ms/step
0.24465082477763644 0.8906250041909516
Epoch 1/1
704/704 [==============================] - 6s 8ms/step
0.17726293171067067 0.9318181853741407
432/432 [==============================] - 3s 6ms/step
0.7055725942556348 0.6435185238994934


array([[193,   9],
       [145,  85]])

In [9]:
data_training = np.concatenate((data_sub_2, data_sub_1, data_sub_3, data_sub_4,
                                data_sub_5, data_sub_6, data_sub_7 , data_sub_8, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_2, labels_sub_1, labels_sub_3, labels_sub_4, 
                                  labels_sub_5, labels_sub_6, labels_sub_7, labels_sub_8, labels_sub_10))


def load_model():
    
    IMG_SIZE = 100
    model = Sequential()
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(100, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation = "sigmoid"))

    model.compile(
     optimizer = "adam",
     loss = "binary_crossentropy",
     metrics = ["accuracy"]
    )
    return model

def train_evaluate(model, data_train, labels_train, data_test, labels_test):
    model.fit(x=data_train, y=labels_train, batch_size=5, epochs=1,
         verbose=1, shuffle=True)
    loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)
    print(loss, acc)

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix


load_model()
kFold = StratifiedKFold(n_splits=5, shuffle = True)
for train, test in kFold.split(data_training,labels_training):
    model = None
    model = load_model()
    train_evaluate(model, data_training[train], labels_training[train], data_training[test], labels_training[test])


loss, acc = model.evaluate(x=data_sub_9, y=labels_sub_9, batch_size=5, verbose=1)
print(loss, acc)
y_pred = model.predict(data_sub_9, verbose=0)
y_pred = np.array([int(round(i[0])) for i in y_pred])
confusion_matrix(labels_sub_9,y_pred)

Epoch 1/1
705/705 [==============================] - 7s 9ms/step
0.4877724530479164 0.7943262501177213
Epoch 1/1
705/705 [==============================] - 7s 9ms/step
0.5294553481095227 0.7035461082948861
Epoch 1/1
705/705 [==============================] - 7s 10ms/step
0.5210294700260704 0.7319149018813532
Epoch 1/1
704/704 [==============================] - 7s 10ms/step
0.5584419684971429 0.6974431917452338
Epoch 1/1
704/704 [==============================] - 7s 10ms/step
0.486122649655127 0.7812500097577206
432/432 [==============================] - 3s 7ms/step
0.5018949090434169 0.7615740848705173


array([[174,  28],
       [ 75, 155]])

In [10]:
data_training = np.concatenate((data_sub_2, data_sub_1, data_sub_3, data_sub_4,
                                data_sub_5, data_sub_6, data_sub_7 , data_sub_8, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_2, labels_sub_1, labels_sub_3, labels_sub_4, 
                                  labels_sub_5, labels_sub_6, labels_sub_7, labels_sub_8, labels_sub_10))


def load_model():
    
    IMG_SIZE = 100
    model = Sequential()
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(100, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation = "sigmoid"))

    model.compile(
     optimizer = "adam",
     loss = "binary_crossentropy",
     metrics = ["accuracy"]
    )
    return model

def train_evaluate(model, data_train, labels_train, data_test, labels_test):
    model.fit(x=data_train, y=labels_train, batch_size=5, epochs=1,
         verbose=1, shuffle=True)
    loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)
    print(loss, acc)

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix


load_model()
kFold = StratifiedKFold(n_splits=5, shuffle = True)
for train, test in kFold.split(data_training,labels_training):
    model = None
    model = load_model()
    train_evaluate(model, data_training[train], labels_training[train], data_training[test], labels_training[test])


loss, acc = model.evaluate(x=data_sub_9, y=labels_sub_9, batch_size=5, verbose=1)
print(loss, acc)
y_pred = model.predict(data_sub_9, verbose=0)
y_pred = np.array([int(round(i[0])) for i in y_pred])
confusion_matrix(labels_sub_9,y_pred)

Epoch 1/1
705/705 [==============================] - 7s 10ms/step
0.5434171909875903 0.5773049653210538
Epoch 1/1
705/705 [==============================] - 7s 11ms/step
0.6252202875647984 0.5773049652153719
Epoch 1/1
705/705 [==============================] - 7s 11ms/step
0.5262450823124419 0.7319149021983992
Epoch 1/1
704/704 [==============================] - 8s 11ms/step
0.5504302204247903 0.7343750070272521
Epoch 1/1
704/704 [==============================] - 8s 11ms/step
0.5591679873918607 0.7997159188274633
432/432 [==============================] - 3s 7ms/step
0.5568248963466397 0.7893518618204527


array([[181,  21],
       [ 70, 160]])

In [13]:
data_training = np.concatenate((data_sub_2, data_sub_1, data_sub_3, data_sub_4,
                                data_sub_5, data_sub_6, data_sub_7 , data_sub_8, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_2, labels_sub_1, labels_sub_3, labels_sub_4, 
                                  labels_sub_5, labels_sub_6, labels_sub_7, labels_sub_8, labels_sub_10))


def load_model():
    
    IMG_SIZE = 100
    model = Sequential()
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(100, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation = "sigmoid"))

    model.compile(
     optimizer = "adam",
     loss = "binary_crossentropy",
     metrics = ["accuracy"]
    )
    return model

def train_evaluate(model, data_train, labels_train, data_test, labels_test):
    model.fit(x=data_train, y=labels_train, batch_size=5, epochs=1,
         verbose=1, shuffle=True)
    loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)
    print(loss, acc)

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix


load_model()
kFold = StratifiedKFold(n_splits=5, shuffle = True)
for train, test in kFold.split(data_training,labels_training):
    model = None
    model = load_model()
    train_evaluate(model, data_training[train], labels_training[train], data_training[test], labels_training[test])


loss, acc = model.evaluate(x=data_sub_9, y=labels_sub_9, batch_size=5, verbose=1)
print(loss, acc)
y_pred = model.predict(data_sub_9, verbose=0)
y_pred = np.array([int(round(i[0])) for i in y_pred])
confusion_matrix(labels_sub_9,y_pred)

Epoch 1/1
705/705 [==============================] - 9s 12ms/step
0.6808978640441353 0.5758865255836054
Epoch 1/1
705/705 [==============================] - 9s 12ms/step
0.6812608859217759 0.5773049651096899
Epoch 1/1
705/705 [==============================] - 9s 13ms/step
0.6258801145756498 0.5773049656380999
Epoch 1/1
704/704 [==============================] - 9s 13ms/step
0.6760460699654438 0.5994318222246048
Epoch 1/1
704/704 [==============================] - 9s 13ms/step
0.6871762769296765 0.5781250004868277
432/432 [==============================] - 3s 7ms/step
0.7472728514974868 0.4675925925925926


array([[202,   0],
       [230,   0]])

## Subject 10

In [ ]:
data_training = np.concatenate((data_sub_2, data_sub_1, data_sub_4, data_sub_5,
                                data_sub_6, data_sub_7, data_sub_8 , data_sub_9, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_2, labels_sub_1, labels_sub_4, labels_sub_5, 
                                  labels_sub_6, labels_sub_7, labels_sub_8, labels_sub_9, labels_sub_10))

limit = math.ceil(len(data_training)/len(data_sub_3))

for i in range(limit):
    data_test = np.concatenate((data_sub_3, data_sub_3)) 
    labels_test = np.concatenate((labels_sub_3, labels_sub_3))
    
IMG_SIZE = 100
model = Sequential()



model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(100, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(50, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(1, activation = "softmax"))

model.compile(
 optimizer = "adam",
 loss = "binary_crossentropy",
 metrics = ["accuracy"]
)

model.fit(x=data_training, y=labels_training, batch_size=5, epochs=3,
     verbose=1, shuffle=True)
loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)

print(loss, acc)

In [ ]:
data_training = np.concatenate((data_sub_2, data_sub_1, data_sub_4, data_sub_5,
                                data_sub_6, data_sub_7, data_sub_8 , data_sub_9, data_sub_10), axis = 0)
labels_training = np.concatenate((labels_sub_2, labels_sub_1, labels_sub_4, labels_sub_5, 
                                  labels_sub_6, labels_sub_7, labels_sub_8, labels_sub_9, labels_sub_10))

limit = math.ceil(len(data_training)/len(data_sub_3))

for i in range(limit):
    data_test = np.concatenate((data_sub_3, data_sub_3)) 
    labels_test = np.concatenate((labels_sub_3, labels_sub_3))
    
IMG_SIZE = 100
model = Sequential()



model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(100, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(50, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(1, activation = "softmax"))

model.compile(
 optimizer = "adam",
 loss = "binary_crossentropy",
 metrics = ["accuracy"]
)

model.fit(x=data_training, y=labels_training, batch_size=5, epochs=3,
     verbose=1, shuffle=True)
loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)

print(loss, acc)

In [14]:
data_training = np.concatenate((data_sub_2, data_sub_1, data_sub_3, data_sub_4,
                                data_sub_5, data_sub_6, data_sub_7 , data_sub_8, data_sub_9), axis = 0)
labels_training = np.concatenate((labels_sub_2, labels_sub_1, labels_sub_3, labels_sub_4, 
                                  labels_sub_5, labels_sub_6, labels_sub_7, labels_sub_8, labels_sub_9))


def load_model():
    
    IMG_SIZE = 100
    model = Sequential()
    
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(100, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation = "sigmoid"))

    model.compile(
     optimizer = "adam",
     loss = "binary_crossentropy",
     metrics = ["accuracy"]
    )
    return model

def train_evaluate(model, data_train, labels_train, data_test, labels_test):
    model.fit(x=data_train, y=labels_train, batch_size=5, epochs=1,
         verbose=1, shuffle=True)
    loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)
    print(loss, acc)

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix


load_model()
kFold = StratifiedKFold(n_splits=5, shuffle = True)
for train, test in kFold.split(data_training,labels_training):
    model = None
    model = load_model()
    train_evaluate(model, data_training[train], labels_training[train], data_training[test], labels_training[test])


loss, acc = model.evaluate(x=data_sub_10, y=labels_sub_10, batch_size=5, verbose=1)
print(loss, acc)
y_pred = model.predict(data_sub_10, verbose=0)
y_pred = np.array([int(round(i[0])) for i in y_pred])
confusion_matrix(labels_sub_10,y_pred)

Epoch 1/1
739/739 [==============================] - 8s 11ms/step
0.24100348597558793 0.8917456076093553
Epoch 1/1
739/739 [==============================] - 8s 11ms/step
0.4216222252132162 0.7848443914411516
Epoch 1/1
739/739 [==============================] - 9s 12ms/step
0.2678469107462678 0.8619756485677056
Epoch 1/1
738/738 [==============================] - 9s 12ms/step
0.23688486632121206 0.8983739890905253
Epoch 1/1
737/737 [==============================] - 9s 12ms/step
0.17461989367417227 0.9294436948432042
263/263 [==============================] - 1s 5ms/step
0.26801446313746696 0.8897338465366074


array([[234,  29],
       [  0,   0]])

In [15]:
data_training = np.concatenate((data_sub_2, data_sub_1, data_sub_3, data_sub_4,
                                data_sub_5, data_sub_6, data_sub_7 , data_sub_8, data_sub_9), axis = 0)
labels_training = np.concatenate((labels_sub_2, labels_sub_1, labels_sub_3, labels_sub_4, 
                                  labels_sub_5, labels_sub_6, labels_sub_7, labels_sub_8, labels_sub_9))


def load_model():
    
    IMG_SIZE = 100
    model = Sequential()
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(100, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation = "sigmoid"))

    model.compile(
     optimizer = "adam",
     loss = "binary_crossentropy",
     metrics = ["accuracy"]
    )
    return model

def train_evaluate(model, data_train, labels_train, data_test, labels_test):
    model.fit(x=data_train, y=labels_train, batch_size=5, epochs=1,
         verbose=1, shuffle=True)
    loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)
    print(loss, acc)

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix


load_model()
kFold = StratifiedKFold(n_splits=5, shuffle = True)
for train, test in kFold.split(data_training,labels_training):
    model = None
    model = load_model()
    train_evaluate(model, data_training[train], labels_training[train], data_training[test], labels_training[test])


loss, acc = model.evaluate(x=data_sub_10, y=labels_sub_10, batch_size=5, verbose=1)
print(loss, acc)
y_pred = model.predict(data_sub_10, verbose=0)
y_pred = np.array([int(round(i[0])) for i in y_pred])
confusion_matrix(labels_sub_10,y_pred)

Epoch 1/1
739/739 [==============================] - 9s 12ms/step
0.4635425429030023 0.790257113167817
Epoch 1/1
739/739 [==============================] - 9s 12ms/step
0.4516641339999989 0.8173207111344124
Epoch 1/1
739/739 [==============================] - 9s 12ms/step
0.4446430984224131 0.8119079911821749
Epoch 1/1
738/738 [==============================] - 9s 13ms/step
0.5038323996634018 0.7831978413672628
Epoch 1/1
737/737 [==============================] - 10s 13ms/step
0.5065444362813574 0.6879240235609472
263/263 [==============================] - 2s 6ms/step
0.3446473052066542 0.8555133160302848


array([[225,  38],
       [  0,   0]])

In [16]:
data_training = np.concatenate((data_sub_2, data_sub_1, data_sub_3, data_sub_4,
                                data_sub_5, data_sub_6, data_sub_7 , data_sub_8, data_sub_9), axis = 0)
labels_training = np.concatenate((labels_sub_2, labels_sub_1, labels_sub_3, labels_sub_4, 
                                  labels_sub_5, labels_sub_6, labels_sub_7, labels_sub_8, labels_sub_9))


def load_model():
    
    IMG_SIZE = 100
    model = Sequential()
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(100, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation = "sigmoid"))

    model.compile(
     optimizer = "adam",
     loss = "binary_crossentropy",
     metrics = ["accuracy"]
    )
    return model

def train_evaluate(model, data_train, labels_train, data_test, labels_test):
    model.fit(x=data_train, y=labels_train, batch_size=5, epochs=1,
         verbose=1, shuffle=True)
    loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)
    print(loss, acc)

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix


load_model()
kFold = StratifiedKFold(n_splits=5, shuffle = True)
for train, test in kFold.split(data_training,labels_training):
    model = None
    model = load_model()
    train_evaluate(model, data_training[train], labels_training[train], data_training[test], labels_training[test])


loss, acc = model.evaluate(x=data_sub_10, y=labels_sub_10, batch_size=5, verbose=1)
print(loss, acc)
y_pred = model.predict(data_sub_10, verbose=0)
y_pred = np.array([int(round(i[0])) for i in y_pred])
confusion_matrix(labels_sub_10,y_pred)

Epoch 1/1
739/739 [==============================] - 10s 13ms/step
0.6736360498186055 0.5548037908396637
Epoch 1/1
739/739 [==============================] - 10s 13ms/step
0.5586559347435328 0.7077131339919744
Epoch 1/1
739/739 [==============================] - 10s 14ms/step
0.6301046496557124 0.6630581947439256
Epoch 1/1
738/738 [==============================] - 10s 14ms/step
0.8318833518864178 0.5352303528486875
Epoch 1/1
737/737 [==============================] - 10s 14ms/step
0.6125653190803657 0.5318860253331781
263/263 [==============================] - 2s 6ms/step
0.5053572570869678 1.0


array([[263]])

In [17]:
data_training = np.concatenate((data_sub_2, data_sub_1, data_sub_3, data_sub_4,
                                data_sub_5, data_sub_6, data_sub_7 , data_sub_8, data_sub_9), axis = 0)
labels_training = np.concatenate((labels_sub_2, labels_sub_1, labels_sub_3, labels_sub_4, 
                                  labels_sub_5, labels_sub_6, labels_sub_7, labels_sub_8, labels_sub_9))


def load_model():
    
    IMG_SIZE = 100
    model = Sequential()
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))

    model.add(Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    model.add(Flatten())
    model.add(Dense(100, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation = "sigmoid"))

    model.compile(
     optimizer = "adam",
     loss = "binary_crossentropy",
     metrics = ["accuracy"]
    )
    return model

def train_evaluate(model, data_train, labels_train, data_test, labels_test):
    model.fit(x=data_train, y=labels_train, batch_size=5, epochs=1,
         verbose=1, shuffle=True)
    loss, acc = model.evaluate(x=data_test, y=labels_test, batch_size=5, verbose=1)
    print(loss, acc)

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix


load_model()
kFold = StratifiedKFold(n_splits=5, shuffle = True)
for train, test in kFold.split(data_training,labels_training):
    model = None
    model = load_model()
    train_evaluate(model, data_training[train], labels_training[train], data_training[test], labels_training[test])


loss, acc = model.evaluate(x=data_sub_10, y=labels_sub_10, batch_size=5, verbose=1)
print(loss, acc)
y_pred = model.predict(data_sub_10, verbose=0)
y_pred = np.array([int(round(i[0])) for i in y_pred])
confusion_matrix(labels_sub_10,y_pred)

Epoch 1/1
739/739 [==============================] - 10s 14ms/step
0.694204522243856 0.5345060905398793
Epoch 1/1
739/739 [==============================] - 11s 14ms/step
0.6709630754706986 0.5290933715756756
Epoch 1/1
739/739 [==============================] - 11s 15ms/step
0.6761167038922703 0.711772675886851
Epoch 1/1
738/738 [==============================] - 11s 15ms/step
0.6888063487402469 0.5352303531515566
Epoch 1/1
737/737 [==============================] - 11s 15ms/step
0.7076733305625709 0.5345997293776702
263/263 [==============================] - 2s 6ms/step
0.4659696926182214 1.0


array([[263]])